In [4]:
#!/usr/bin/env python3
"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║     SCIENTIFIC OPTIONS FRAMEWORK v4.9 - TRUTH MODE                                               ║
║     ════════════════════════════════════════════════════════════════════════                    ║
║                                                                                                  ║
║     v4.9 TRUTH MODE - REALISTIC CALIBRATION FOR HONEST RESULTS                                  ║
║     ─────────────────────────────────────────────────────────────                               ║
║     • Greeks recalibrated: Theta cut ~50%, Gamma penalty increased                              ║
║     • Scale factors reduced: GAMMA_SCALE=8 (was 12), VEGA_SCALE=6 (was 10)                     ║
║     • Asset-specific Greek multipliers (BTC=2.2x, FX=0.6x, etc.)                               ║
║     • Transaction costs increased to realistic levels                                            ║
║     • Tighter ex-ante gates (vol_of_vol=0.25, extreme_vol=0.995)                               ║
║                                                                                                  ║
║     EXPECTED TRUTH MODE RESULTS:                                                                 ║
║     • Sharpe: 0.8 - 1.3 (was 4-5 in "lab mode")                                                ║
║     • Sortino: 1.5 - 2.2                                                                        ║
║     • MaxDD: -15% to -25%                                                                       ║
║                                                                                                  ║
║     This exposes whether strategy has REAL structural alpha vs parameter-driven upside.         ║
║                                                                                                  ║
║     v4.8 FIX RETAINED: No look-ahead bias (ex-ante gates only)                                  ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import norm, skew, kurtosis, probplot, t as t_dist
from scipy import stats
from scipy.optimize import minimize
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any, Union
from dataclasses import dataclass, field
from pathlib import Path
from datetime import datetime, timedelta
from enum import Enum
import warnings
import sys
import copy  # v4.1: Added for deepcopy support

warnings.filterwarnings("ignore")

# =============================================================================
# VERSION & CONSTANTS
# =============================================================================

FRAMEWORK_VERSION = "4.9"
FRAMEWORK_NAME = f"Scientific Options Framework v{FRAMEWORK_VERSION}"


# =============================================================================
# OPTIONS GREEKS PNL MODEL (v4.9 TRUTH MODE)
# =============================================================================

class OptionsGreeksPnLModel:
    """
    Proper options premium selling PnL model based on Greeks.

    v4.9 TRUTH MODE - REALISTIC CALIBRATION:
    ─────────────────────────────────────────
    • Theta edge cut ~50% (realistic premium decay)
    • Gamma penalty increased (realistic convexity risk)
    • Vega sensitivity lowered (no fake vol edge)
    • Scale factors reduced: GAMMA=8, VEGA=6

    EXPECTED TRUTH MODE RESULTS:
    • Sharpe: 0.8 - 1.3 (was 4-5 in "lab mode")
    • Sortino: 1.5 - 2.2
    • MaxDD: -18% to -25%

    This exposes whether strategy has REAL structural alpha.
    """

    # v4.9 TRUTH MODE GREEKS - Realistic calibration
    # Theta cut ~50%, Gamma increased, more conservative
    STRATEGY_GREEKS = {
        'Strangle':       {'theta': 0.0007, 'vega': 0.045, 'gamma': -0.040, 'delta': 0.0},
        'IronCondor':     {'theta': 0.0005, 'vega': 0.032, 'gamma': -0.030, 'delta': 0.0},
        'ShortPut':       {'theta': 0.0004, 'vega': 0.028, 'gamma': -0.033, 'delta': 0.12},
        'ShortCall':      {'theta': 0.0004, 'vega': 0.028, 'gamma': -0.033, 'delta': -0.12},
        'BullPutSpread':  {'theta': 0.0003, 'vega': 0.018, 'gamma': -0.015, 'delta': 0.08},
        'BearCallSpread': {'theta': 0.0003, 'vega': 0.018, 'gamma': -0.015, 'delta': -0.08},
        'Cash':           {'theta': 0.0, 'vega': 0.0, 'gamma': 0.0, 'delta': 0.0},
    }

    # v4.9 TRUTH MODE Scale factors (reduced from 12/10)
    GAMMA_SCALE = 8.0
    VEGA_SCALE = 6.0

    # v4.9: Asset-specific Greek multipliers for cross-asset realism
    ASSET_GREEK_SCALARS = {
        # Equities (base)
        "USA500.IDXUSD": 1.0,
        "USA30.IDXUSD": 1.0,
        "USATECH.IDXUSD": 1.1,
        "USSC2000.IDXUSD": 1.15,
        # Crypto (much higher vol)
        "BTCUSD": 2.2,
        "ETHUSD": 2.5,
        # Precious metals
        "XAUUSD": 0.9,
        "XAGUSD": 1.1,
        # Energy (high vol)
        "GAS.CMDUSD": 1.6,
        "BRENT.CMDUSD": 1.3,
        "LIGHT.CMDUSD": 1.3,
        "DIESEL.CMDUSD": 1.4,
        # FX (lower vol)
        "EURUSD": 0.6,
        "GBPUSD": 0.65,
        "USDJPY": 0.5,
        "AUDUSD": 0.7,
        "USDCAD": 0.6,
        "USDCHF": 0.55,
        "NZDUSD": 0.7,
        # Bonds
        "USTBOND.TRUSD": 0.8,
        "UKGILT.TRGBP": 0.8,
        # Softs/Ags
        "COFFEE.CMDUSX": 1.3,
        "COCOA.CMDUSX": 1.3,
        "SUGAR.CMDUSD": 1.2,
        "COTTON.CMDUSX": 1.2,
        "SOYBEAN.CMDUSX": 1.1,
        "OJUICE.CMDUSX": 1.3,
        # Metals
        "COPPER.CMDUSD": 1.1,
    }

    @classmethod
    def get_asset_scalar(cls, ticker: str) -> float:
        """Get asset-specific Greek multiplier for realism."""
        return cls.ASSET_GREEK_SCALARS.get(ticker, 1.0)

    @classmethod
    def calculate_pnl(cls, strategy: str, position_size: float,
                      price_return: float, vol_change: float,
                      days: float = 1.0, ticker: str = None) -> dict:
        """
        Calculate options strategy PnL using Greeks model.

        v4.9 TRUTH MODE:
        - Asset-specific Greek scaling for cross-asset realism
        - Reduced scale factors (GAMMA=8, VEGA=6)
        - More realistic theta/gamma balance
        """
        greeks = cls.STRATEGY_GREEKS.get(strategy, cls.STRATEGY_GREEKS['Cash'])

        # v4.9: Asset-specific scaling
        asset_scalar = cls.get_asset_scalar(ticker) if ticker else 1.0

        # Scale by position size AND asset scalar
        theta = greeks['theta'] * position_size
        vega = greeks['vega'] * position_size * asset_scalar
        gamma = greeks['gamma'] * position_size * asset_scalar
        delta = greeks['delta'] * position_size

        # Vega PnL: Convert ANN vol to daily IV basis
        iv_change = vol_change / np.sqrt(252)
        vega_pnl = -vega * iv_change * cls.VEGA_SCALE

        # Gamma PnL: Quadratic in returns, v4.9 uses 8x scale
        gamma_pnl = 0.5 * gamma * (price_return ** 2) * cls.GAMMA_SCALE

        # Theta and Delta
        theta_pnl = theta * days
        delta_pnl = delta * price_return

        total_pnl = theta_pnl + vega_pnl + gamma_pnl + delta_pnl

        return {
            'theta_pnl': theta_pnl,
            'vega_pnl': vega_pnl,
            'gamma_pnl': gamma_pnl,
            'delta_pnl': delta_pnl,
            'total_pnl': total_pnl
        }

    @classmethod
    def gamma_exceeds_gains(cls, theta_pnl: float, vega_pnl: float,
                            gamma_pnl: float, threshold: float = 1.5) -> bool:
        """
        DEPRECATED in v4.8+ - HAD LOOK-AHEAD BIAS!
        Kept for backwards compatibility only.
        """
        import warnings
        warnings.warn(
            "gamma_exceeds_gains() is deprecated due to look-ahead bias.",
            DeprecationWarning
        )
        gains = theta_pnl + max(vega_pnl, 0)
        return gamma_pnl < -(gains * threshold)

# Centralized random seeding for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

def set_random_seed(seed: int = 42):
    """Set random seed for reproducibility across all components."""
    global RANDOM_SEED
    RANDOM_SEED = seed
    np.random.seed(seed)

# =============================================================================
# WIRING STATUS (What's actually implemented)
# =============================================================================
#
# WIRED INTO BACKTEST (ScientificStrategyEngine.run):
#   • HMMRegimeDetector - Regime detection with proper train/test split
#   • MLSignalGenerator - ML signals with OOS AUC tracking
#   • VolPercentileEngine - Realized vol percentile (training distribution)
#   • WeekendThetaConfig - PnL adjustment for Friday positions
#   • StrategyLeverageConfig - Per-strategy position sizing
#   • Transaction costs - Applied on strategy changes
#   • ART vol targeting - Position scaling based on realized vol
#
# NOT WIRED (FUTURE WORK / REMOVED):
#   • Heston stochastic vol - Parameters removed (use empirical vol instead)
#   • Greek-based delta hedging - Not implemented (no option pricer)
#   • Path-dependent stops/profit targets - Not implemented
#   • Capital constraints - Position is dimensionless signal, not dollars
#

# =============================================================================
# COLOR-CODED LOGGING (Preserved from v3)
# =============================================================================

class Colors:
    """ANSI color codes for terminal output."""
    RESET = '\033[0m'
    BOLD = '\033[1m'

    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'

    BRIGHT_RED = '\033[91m'
    BRIGHT_GREEN = '\033[92m'
    BRIGHT_YELLOW = '\033[93m'
    BRIGHT_BLUE = '\033[94m'
    BRIGHT_MAGENTA = '\033[95m'
    BRIGHT_CYAN = '\033[96m'

    @staticmethod
    def disable():
        for attr in ['RESET', 'BOLD', 'RED', 'GREEN', 'YELLOW', 'BLUE',
                     'MAGENTA', 'CYAN', 'WHITE', 'BLACK',
                     'BRIGHT_RED', 'BRIGHT_GREEN', 'BRIGHT_YELLOW',
                     'BRIGHT_BLUE', 'BRIGHT_MAGENTA', 'BRIGHT_CYAN']:
            setattr(Colors, attr, '')


class LogLevel(Enum):
    """Logging verbosity levels."""
    QUIET = 0
    NORMAL = 1
    VERBOSE = 2
    DEBUG = 3


@dataclass
class LoggingConfig:
    """Logging configuration."""
    level: LogLevel = LogLevel.NORMAL
    log_positions: bool = True
    log_costs: bool = True
    log_turnover: bool = True
    log_greeks: bool = False
    log_signals: bool = False
    log_pnl: bool = True
    use_colors: bool = True
    show_progress: bool = True
    summary_frequency: int = 50
    save_to_file: bool = False
    log_file_path: Optional[Path] = None

    def __post_init__(self):
        if not self.use_colors or not sys.stdout.isatty():
            Colors.disable()


class Logger:
    """Centralized logger with color-coded output."""

    def __init__(self, config: LoggingConfig):
        self.config = config
        self.log_file = None
        if config.save_to_file and config.log_file_path:
            self.log_file = open(config.log_file_path, 'w')

    def _should_log(self, required_level: LogLevel) -> bool:
        return self.config.level.value >= required_level.value

    def _log(self, message: str, color: str = '', to_file: bool = True):
        colored_msg = f"{color}{message}{Colors.RESET}"
        print(colored_msg)
        if to_file and self.log_file:
            self.log_file.write(message + '\n')
            self.log_file.flush()

    def header(self, message: str):
        if self._should_log(LogLevel.NORMAL):
            self._log(f"\n{'='*80}\n{message}\n{'='*80}", Colors.BOLD + Colors.CYAN)

    def info(self, message: str):
        if self._should_log(LogLevel.NORMAL):
            self._log(f"ℹ {message}", Colors.BLUE)

    def success(self, message: str):
        if self._should_log(LogLevel.NORMAL):
            self._log(f"✓ {message}", Colors.BRIGHT_GREEN)

    def warning(self, message: str):
        if self._should_log(LogLevel.NORMAL):
            self._log(f"⚠ {message}", Colors.BRIGHT_YELLOW)

    def error(self, message: str):
        self._log(f"✗ {message}", Colors.BRIGHT_RED)

    def debug(self, message: str):
        if self._should_log(LogLevel.DEBUG):
            self._log(f"🐛 {message}", Colors.MAGENTA)

    def position_open(self, strategy: str, size: float, price: float, cost: float):
        if self.config.log_positions and self._should_log(LogLevel.VERBOSE):
            msg = f"OPEN {strategy}: size={size:.2f}, price={price:.4f}, cost=${cost:.2f}"
            self._log(msg, Colors.GREEN)

    def position_close(self, strategy: str, reason: str, pnl: float, holding_days: int):
        if self.config.log_positions and self._should_log(LogLevel.VERBOSE):
            color = Colors.BRIGHT_GREEN if pnl > 0 else Colors.BRIGHT_RED
            msg = f"CLOSE {strategy} ({reason}): P&L=${pnl:.2f}, held {holding_days} days"
            self._log(msg, color)

    def progress(self, current: int, total: int, desc: str = ""):
        if self.config.show_progress and self._should_log(LogLevel.NORMAL):
            pct = 100 * current / total if total > 0 else 0
            bar_len = 40
            filled = int(bar_len * current / total) if total > 0 else 0
            bar = '█' * filled + '░' * (bar_len - filled)
            msg = f"{desc} [{bar}] {pct:.1f}% ({current}/{total})"
            print(f"\r{Colors.BLUE}{msg}{Colors.RESET}", end='', flush=True)
            if current == total:
                print()

    def summary(self, metrics: Dict[str, float]):
        if self._should_log(LogLevel.NORMAL):
            self._log("\n" + "─"*80, Colors.CYAN)
            self._log("SUMMARY STATISTICS", Colors.BOLD + Colors.CYAN)
            self._log("─"*80, Colors.CYAN)
            for key, value in metrics.items():
                if isinstance(value, float):
                    if 'pct' in key.lower() or 'rate' in key.lower():
                        msg = f"  {key:.<40} {value:>10.2%}"
                    else:
                        msg = f"  {key:.<40} {value:>10.4f}"
                else:
                    msg = f"  {key:.<40} {value:>10}"
                color = Colors.GREEN if isinstance(value, (int, float)) and value > 0 else \
                        Colors.RED if isinstance(value, (int, float)) and value < 0 else Colors.WHITE
                self._log(msg, color)
            self._log("─"*80 + "\n", Colors.CYAN)

    def close(self):
        if self.log_file:
            self.log_file.close()


# =============================================================================
# THEME DEFINITIONS (Preserved from v3)
# =============================================================================

class DarkTheme:
    """Dark theme color palette."""
    BACKGROUND = '#1a1a2e'
    PANEL = '#16213e'
    BLUE = '#4da6ff'
    ORANGE = '#ff9f43'
    GREEN = '#00d4aa'
    RED = '#ff6b6b'
    PURPLE = '#a855f7'
    CYAN = '#00d4ff'
    PINK = '#ff69b4'
    HISTOGRAM = '#4a90d9'
    GRID = '#3a3a5a'
    TEXT = '#e0e0e0'
    TEXT_DIM = '#a0a0a0'
    GOLD = '#ffd700'
    YELLOW = '#ffc107'


class LightTheme:
    """Light theme color palette."""
    PRIMARY = '#1a3a5e'
    SECONDARY = '#6a33f5'
    ACCENT = '#00d4aa'
    POSITIVE = '#00c853'
    NEGATIVE = '#ff1744'
    WARNING = '#ffc107'
    NEUTRAL = '#78909c'
    GOLD = '#ffd700'


def setup_dark_theme():
    """Apply dark theme to matplotlib."""
    plt.rcParams.update({
        'figure.facecolor': DarkTheme.BACKGROUND,
        'axes.facecolor': DarkTheme.PANEL,
        'axes.edgecolor': DarkTheme.GRID,
        'axes.labelcolor': DarkTheme.TEXT,
        'text.color': DarkTheme.TEXT,
        'xtick.color': DarkTheme.TEXT,
        'ytick.color': DarkTheme.TEXT,
        'grid.color': DarkTheme.GRID,
        'grid.alpha': 0.3,
        'legend.facecolor': DarkTheme.PANEL,
        'legend.edgecolor': DarkTheme.GRID,
        'legend.labelcolor': DarkTheme.TEXT,
    })


def reset_light_theme():
    """Reset to light theme."""
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams.update({
        'font.size': 10,
        'axes.titlesize': 14,
        'axes.labelsize': 11,
        'figure.facecolor': 'white',
        'axes.facecolor': 'white',
        'axes.spines.top': False,
        'axes.spines.right': False,
    })


# =============================================================================
# WEEKEND THETA CONFIGURATION (NOW ACTUALLY WIRED - FIX #2)
# =============================================================================

@dataclass
class WeekendThetaConfig:
    """Weekend theta decay configuration - NOW ACTUALLY AFFECTS PNL."""
    enabled: bool = True
    weekend_factor: float = 3.0  # Base factor for Fri→Mon decay
    dte_scaling_enabled: bool = True
    dte_7_factor: float = 3.5    # Weeklies: more theta decay
    dte_14_factor: float = 3.0   # 2-week options
    dte_30_factor: float = 2.5   # Monthlies: less theta impact
    long_weekend_factor: float = 4.0  # Holidays
    test_mode: bool = False
    test_factors: List[float] = field(default_factory=lambda: [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0])

    def get_weekend_factor(self, dte: int = 7, is_long_weekend: bool = False) -> float:
        """Get weekend decay factor based on DTE."""
        if not self.enabled:
            return 1.0
        if is_long_weekend:
            return self.long_weekend_factor
        if not self.dte_scaling_enabled:
            return self.weekend_factor
        if dte <= 7:
            return self.dte_7_factor
        elif dte <= 14:
            return self.dte_14_factor
        elif dte <= 30:
            return self.dte_30_factor
        else:
            return 2.0


# =============================================================================
# STRATEGY LEVERAGE CONFIG (FIX #4 - Different leverage per strategy)
# =============================================================================

@dataclass
class StrategyLeverageConfig:
    """
    Different leverage/scaling per option strategy type.

    Rationale:
    - Short strangles: higher leverage (selling premium both sides)
    - Iron condors: moderate (defined risk)
    - Vertical spreads: lower (single direction, defined risk)
    - Naked puts/calls: high leverage but capped
    """
    bull_put_spread: float = 1.0      # Defined risk, moderate
    bear_call_spread: float = 1.0     # Defined risk, moderate
    iron_condor: float = 1.5          # Premium collection both sides
    short_put: float = 1.2            # Naked, capped
    short_call: float = 1.2           # Naked, capped
    strangle: float = 1.8             # Most aggressive premium selling
    cash: float = 0.0                 # No position

    def get_leverage(self, strategy: str) -> float:
        """Get leverage multiplier for strategy."""
        leverage_map = {
            'BullPutSpread': self.bull_put_spread,
            'BearCallSpread': self.bear_call_spread,
            'IronCondor': self.iron_condor,
            'ShortPut': self.short_put,
            'ShortCall': self.short_call,
            'Strangle': self.strangle,
            'Cash': self.cash
        }
        return leverage_map.get(strategy, 1.0)


# =============================================================================
# ENHANCED CONFIGURATION
# =============================================================================

@dataclass
class ScientificConfig:
    """
    Configuration for Scientific Options Framework v4.9 TRUTH MODE.

    v4.9 TRUTH MODE - REALISTIC CALIBRATION:
    ─────────────────────────────────────────
    • Lower target_vol (0.08)
    • Higher transaction costs
    • Tighter drawdown limits
    • Stricter ex-ante gates
    • Reduced position sizes

    GOAL: Expose real structural alpha, not parameter-driven upside.
    """

    # Paths
    data_dir: Path = field(default_factory=lambda: Path("/Users/jasongu/Downloads/DataSet2"))
    report_dir: Path = field(default_factory=lambda: Path("/Users/jasongu/Downloads/DataSet2/Reports"))

    # Logging
    logging: LoggingConfig = field(default_factory=LoggingConfig)

    # Weekend theta (WIRED into PnL)
    weekend_theta: WeekendThetaConfig = field(default_factory=WeekendThetaConfig)

    # Strategy leverage (WIRED into position sizing)
    strategy_leverage: StrategyLeverageConfig = field(default_factory=StrategyLeverageConfig)

    # Account (for equity curve tracking, not actual capital constraints)
    account_size: float = 100_000  # Initial notional for equity curve

    # Walk-forward (ACTIVELY USED)
    train_window: int = 252 * 2  # 2 years
    test_window: int = 21        # 1 month
    min_train_size: int = 252

    # ML (ACTIVELY USED) - v4.9 TRUTH MODE: More conservative
    ml_regularization: float = 0.1
    ml_model_type: str = 'gradient_boosting'
    ml_n_estimators: int = 80   # v4.9: Reduced from 100
    ml_max_depth: int = 3       # v4.9: Shallower trees (was 4)
    ml_learning_rate: float = 0.03  # v4.9: Reduced from 0.05
    ml_min_probability: float = 0.58  # v4.9: Higher bar (was 0.55)
    ml_features: List[str] = field(default_factory=lambda: [
        'ret_1', 'ret_5', 'vol_20', 'vol_parkinson', 'vol_gk',
        'rsi_14', 'momentum', 'price_to_sma_20',
        'vol_percentile', 'vol_of_vol', 'atr_14',
        'skew_20_shrunk', 'kurt_20_shrunk'
    ])

    # HMM (ACTIVELY USED)
    hmm_n_states: int = 3
    hmm_n_iter: int = 100

    # Vol Percentile - v4.9 TRUTH MODE: More selective
    vol_percentile_threshold: float = 0.70  # v4.9: Slightly less selective (was 0.75)
    vol_percentile_lookback: int = 252
    extreme_vol_cutoff: float = 0.995  # v4.9: Only TRUE crises (was 0.98)

    # Risk - v4.9 TRUTH MODE: More conservative
    target_vol: float = 0.08  # v4.9: Realistic fund level (was 0.10)
    var_confidence: float = 0.95
    max_drawdown_limit: float = -0.20  # For reporting
    max_position_size: float = 0.25  # v4.9: Reduced leverage (was 0.40)
    vol_smoothing_window: int = 60

    # v4.9 TRUTH MODE: Tighter drawdown discipline
    dd_half_size_threshold: float = -0.07  # v4.9: Tighter (was -0.10)
    dd_flat_threshold: float = -0.15  # v4.9: Tighter (was -0.20)

    # v4.9 TRUTH MODE: Tighter ex-ante gates
    large_move_threshold: float = 2.5  # v4.9: Tighter (was 2.0)
    vol_of_vol_threshold: float = 0.25  # v4.9: Vol must REALLY explode (was 0.15)

    # Turnover control - v4.9: More reactive
    strategy_rebalance_days: int = 3  # v4.9: More reactive (was 5)
    atr_sideways_threshold: float = 0.002

    # Options (used for DTE in weekend theta)
    risk_free_rate: float = 0.05
    dte_weekly: int = 7
    dte_monthly: int = 30
    use_weekly: bool = True

    # Transaction costs - v4.9 TRUTH MODE: Higher realistic costs
    tx_commission: float = 0.0008  # v4.9: Higher (was 0.0005*0.95)
    tx_spread: float = 0.0012     # v4.9: Higher (was 0.001*0.95)
    tx_slippage: float = 0.0007   # v4.9: Higher (was 0.0005*0.95)

    # Forecast (for analysis columns)
    forecast_horizon: int = 5

    # Reliability diagram settings
    reliability_n_bins: int = 10

    # Higher moment shrinkage (ACTIVELY USED)
    moment_shrinkage_lambda: float = 0.5

    def __post_init__(self):
        self.report_dir.mkdir(parents=True, exist_ok=True)
        if self.logging.save_to_file and self.logging.log_file_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            self.logging.log_file_path = self.report_dir / f"backtest_{timestamp}.log"

    def copy(self) -> 'ScientificConfig':
        """
        Create a deep copy of the configuration.

        v4.1 NEW: Useful for parameter sweeps and testing.
        """
        import copy
        return copy.deepcopy(self)

    def with_overrides(self, **kwargs) -> 'ScientificConfig':
        """
        Create a copy with specific overrides.

        v4.1 NEW: Convenient for parameter testing.

        Example:
            test_config = config.with_overrides(
                train_window=504,
                target_vol=0.15
            )
        """
        import copy
        new_config = copy.deepcopy(self)

        for key, value in kwargs.items():
            if hasattr(new_config, key):
                setattr(new_config, key, value)
            elif '.' in key:
                # Handle nested attributes like 'weekend_theta.weekend_factor'
                parts = key.split('.', 1)
                if hasattr(new_config, parts[0]):
                    parent = getattr(new_config, parts[0])
                    if hasattr(parent, parts[1]):
                        setattr(parent, parts[1], value)
            else:
                raise AttributeError(f"ScientificConfig has no attribute '{key}'")

        return new_config


# =============================================================================
# ENHANCED DATA LOADER (NEW: Parkinson, GK vol, ATR, shrunk moments)
# =============================================================================

class DataLoader:
    """Load and preprocess market data with enhanced features."""

    def __init__(self, config: ScientificConfig):
        self.config = config

    def load(self, ticker: str) -> pd.DataFrame:
        """Load data for a ticker."""
        files = list(self.config.data_dir.glob(f"*{ticker.split('.')[0]}*.csv"))
        if not files:
            raise FileNotFoundError(f"No data file found for {ticker}")

        df = pd.read_csv(files[0], parse_dates=['timestamp'])
        df.set_index('timestamp', inplace=True)

        daily = df.resample('D').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()

        return daily

    def add_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add technical features including new estimators."""
        df = df.copy()

        # =====================================================================
        # BASIC RETURNS
        # =====================================================================
        df['ret_1'] = df['close'].pct_change(1)
        df['ret_5'] = df['close'].pct_change(5)
        df['ret_20'] = df['close'].pct_change(20)

        # =====================================================================
        # VOLATILITY ESTIMATORS (ENHANCED)
        # =====================================================================

        # 1. Close-to-close volatility (original)
        df['vol_20'] = df['ret_1'].rolling(20).std() * np.sqrt(252)
        df['vol_60'] = df['ret_1'].rolling(60).std() * np.sqrt(252)

        # 2. Parkinson volatility (uses high-low range) - NEW
        # More efficient than close-to-close for continuous processes
        log_hl = np.log(df['high'] / df['low'])
        df['vol_parkinson'] = np.sqrt(
            (1 / (4 * np.log(2))) * (log_hl ** 2).rolling(20).mean() * 252
        )

        # 3. Garman-Klass volatility (uses OHLC) - NEW
        # Most efficient estimator using all OHLC data
        log_hl = np.log(df['high'] / df['low'])
        log_co = np.log(df['close'] / df['open'])
        df['vol_gk'] = np.sqrt(
            (0.5 * log_hl**2 - (2*np.log(2) - 1) * log_co**2).rolling(20).mean() * 252
        )

        # 4. ATR (Average True Range) - NEW
        high_low = df['high'] - df['low']
        high_close = np.abs(df['high'] - df['close'].shift(1))
        low_close = np.abs(df['low'] - df['close'].shift(1))
        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr_14'] = true_range.rolling(14).mean()
        df['atr_pct'] = df['atr_14'] / df['close']  # ATR as % of price

        # 5. Vol of vol (preserved)
        df['vol_of_vol'] = df['vol_20'].rolling(20).std()

        # =====================================================================
        # MOMENTUM & TREND
        # =====================================================================

        # RSI
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss.replace(0, np.nan)
        df['rsi_14'] = 100 - (100 / (1 + rs))

        # Momentum
        df['momentum'] = df['close'] / df['close'].shift(20) - 1

        # SMA
        df['sma_20'] = df['close'].rolling(20).mean()
        df['price_to_sma_20'] = df['close'] / df['sma_20'] - 1

        # =====================================================================
        # HIGHER MOMENTS WITH SHRINKAGE (FIX - more robust)
        # =====================================================================

        # Raw moments
        df['skew_20_raw'] = df['ret_1'].rolling(20).apply(skew, raw=True)
        df['kurt_20_raw'] = df['ret_1'].rolling(20).apply(kurtosis, raw=True)

        # Shrunk moments (toward 0 for skew, toward 0 for excess kurtosis)
        # Formula: shrunk = raw / sqrt(1 + lambda)
        shrink_factor = np.sqrt(1 + self.config.moment_shrinkage_lambda)
        df['skew_20_shrunk'] = df['skew_20_raw'] / shrink_factor
        df['kurt_20_shrunk'] = df['kurt_20_raw'] / shrink_factor

        # Also compute on longer window (more stable)
        df['skew_60'] = df['ret_1'].rolling(60).apply(skew, raw=True)
        df['kurt_60'] = df['ret_1'].rolling(60).apply(kurtosis, raw=True)

        # =====================================================================
        # DAY OF WEEK (for weekend theta)
        # =====================================================================
        df['day_of_week'] = df.index.dayofweek  # 0=Monday, 4=Friday
        df['is_friday'] = (df['day_of_week'] == 4).astype(int)

        return df.dropna()


# =============================================================================
# HMM REGIME DETECTOR (FIX #1 - No more leakage)
# =============================================================================

class HMMRegimeDetector:
    """
    Hidden Markov Model for regime detection.

    FIX #1 (v4.0): Now properly separates fit (train) from predict (test).
    FIX 1.2 (v4.2): Maps states to Bull/Neutral/Bear based on actual return/vol properties.
    """

    def __init__(self, n_states: int = 3, n_iter: int = 200):
        self.n_states = n_states
        self.n_iter = n_iter
        self.model = None
        self.state_names = ['Bull', 'Neutral', 'Bear'][:n_states]
        self._feature_mean = None
        self._feature_std = None
        # v4.2: State mapping based on properties
        self._state_mapping = None  # Maps raw state index to semantic name
        self._state_stats = {}  # Statistics per state

    def _prepare_features(self, df: pd.DataFrame) -> np.ndarray:
        """Prepare feature matrix for HMM."""
        features = []
        for col in ['vol_20', 'vol_of_vol', 'skew_20_shrunk', 'ret_5']:
            if col in df.columns:
                features.append(df[col].fillna(method='ffill').fillna(0).values)

        if not features:
            features = [df['ret_1'].fillna(0).values]

        return np.column_stack(features)

    def _compute_state_mapping(self, train_df: pd.DataFrame, raw_states: np.ndarray):
        """
        FIX 1.2: Compute mapping from raw HMM states to semantic regime names.

        Maps states based on mean return:
        - Highest mean return → Bull (state 0)
        - Middle mean return → Neutral (state 1)
        - Lowest mean return → Bear (state 2)

        This ensures regime labels match actual market behavior.
        """
        if 'ret_1' not in train_df.columns:
            # Fallback: use raw states as-is
            self._state_mapping = {i: i for i in range(self.n_states)}
            return

        returns = train_df['ret_1'].values
        vol = train_df['vol_20'].values if 'vol_20' in train_df.columns else np.ones(len(train_df)) * 0.15

        # Compute per-state statistics
        state_stats = []
        for state in range(self.n_states):
            mask = raw_states == state
            if mask.sum() > 0:
                mean_ret = np.nanmean(returns[mask])
                mean_vol = np.nanmean(vol[mask])
                count = mask.sum()
            else:
                mean_ret = 0.0
                mean_vol = 0.15
                count = 0

            state_stats.append({
                'raw_state': state,
                'mean_return': mean_ret,
                'mean_vol': mean_vol,
                'count': count
            })

        # Sort by mean return (descending) to assign semantic names
        sorted_stats = sorted(state_stats, key=lambda x: x['mean_return'], reverse=True)

        # Create mapping: raw_state → semantic_state
        # Highest return = Bull (0), Middle = Neutral (1), Lowest = Bear (2)
        self._state_mapping = {}
        semantic_names = ['Bull', 'Neutral', 'Bear']

        for semantic_idx, stats in enumerate(sorted_stats):
            raw_state = stats['raw_state']
            self._state_mapping[raw_state] = semantic_idx
            self._state_stats[semantic_idx] = {
                'name': semantic_names[min(semantic_idx, len(semantic_names)-1)],
                'mean_return': stats['mean_return'],
                'mean_vol': stats['mean_vol'],
                'count': stats['count']
            }

        # Update state_names to reflect actual mapping
        self.state_names = [semantic_names[i] for i in range(min(self.n_states, 3))]

    def _apply_state_mapping(self, raw_states: np.ndarray) -> np.ndarray:
        """Apply state mapping to convert raw HMM states to semantic states."""
        if self._state_mapping is None:
            return raw_states

        mapped = np.zeros_like(raw_states)
        for raw, semantic in self._state_mapping.items():
            mapped[raw_states == raw] = semantic
        return mapped

    def get_state_statistics(self) -> Dict[int, Dict]:
        """Return computed statistics per regime for analysis/debugging."""
        return self._state_stats

    def fit(self, train_df: pd.DataFrame) -> 'HMMRegimeDetector':
        """
        Fit HMM on training data ONLY.

        FIX #1: Only uses training data to learn the model.
        FIX 1.2 (v4.2): Computes state→regime mapping based on return/vol properties.
        """
        try:
            from hmmlearn import hmm

            X = self._prepare_features(train_df)

            # Store normalization parameters from training data
            self._feature_mean = np.nanmean(X, axis=0)
            self._feature_std = np.nanstd(X, axis=0) + 1e-8

            # Normalize
            X_norm = (X - self._feature_mean) / self._feature_std
            X_norm = np.nan_to_num(X_norm, nan=0, posinf=0, neginf=0)

            # Fit model
            self.model = hmm.GaussianHMM(
                n_components=self.n_states,
                covariance_type='full',
                n_iter=self.n_iter,
                random_state=42
            )
            self.model.fit(X_norm)

            # FIX 1.2: Compute state mapping based on actual return/vol properties
            raw_states = self.model.predict(X_norm)
            self._compute_state_mapping(train_df, raw_states)

        except ImportError:
            # Fallback: store volatility percentiles from training
            vol = train_df['vol_20'].fillna(method='ffill').values if 'vol_20' in train_df.columns else np.ones(len(train_df)) * 0.15
            self._vol_p33 = np.percentile(vol, 33)
            self._vol_p67 = np.percentile(vol, 67)
            self.model = None
            # Fallback mapping: low vol = Bull, high vol = Bear
            self._state_mapping = {0: 0, 1: 1, 2: 2}

        return self

    def predict(self, test_df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Predict states on test data using model fitted on training data.

        No future information leakage.
        FIX 1.2 (v4.2): Returns semantically-mapped states (Bull=0, Neutral=1, Bear=2).
        """
        try:
            from hmmlearn import hmm

            if self.model is None:
                raise ImportError("Model not fitted")

            X = self._prepare_features(test_df)

            # Use training normalization parameters (not test statistics!)
            X_norm = (X - self._feature_mean) / self._feature_std
            X_norm = np.nan_to_num(X_norm, nan=0, posinf=0, neginf=0)

            raw_states = self.model.predict(X_norm)
            raw_probs = self.model.predict_proba(X_norm)

            # FIX 1.2: Apply semantic mapping so Bull=0, Neutral=1, Bear=2
            states = self._apply_state_mapping(raw_states)

            # Reorder probability columns to match mapped states
            probs = np.zeros_like(raw_probs)
            if self._state_mapping is not None:
                for raw, semantic in self._state_mapping.items():
                    probs[:, semantic] = raw_probs[:, raw]
            else:
                probs = raw_probs

            return states, probs

        except (ImportError, AttributeError):
            # Fallback using training percentiles
            # FIX 2.4: Guard against missing attributes from failed fit()
            if not hasattr(self, '_vol_p33') or not hasattr(self, '_vol_p67'):
                # Safe defaults if percentiles weren't computed
                self._vol_p33 = 0.10
                self._vol_p67 = 0.20

            vol = test_df['vol_20'].fillna(method='ffill').values if 'vol_20' in test_df.columns else np.ones(len(test_df)) * 0.15

            states = np.ones(len(test_df), dtype=int)  # Default neutral
            states[vol < self._vol_p33] = 0  # Bull (low vol)
            states[vol >= self._vol_p67] = 2  # Bear (high vol)

            probs = np.zeros((len(test_df), self.n_states))
            for i, s in enumerate(states):
                probs[i, s] = 0.7
                for j in range(self.n_states):
                    if j != s:
                        probs[i, j] = 0.3 / (self.n_states - 1)

            return states, probs

    def fit_predict(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Legacy method for single-pass (used for training visualization).
        For proper walk-forward, use fit() then predict() separately.
        """
        self.fit(df)
        return self.predict(df)


# =============================================================================
# ML SIGNAL GENERATOR (FIX #3 - OOS AUC, not train AUC)
# =============================================================================

class MLSignalGenerator:
    """
    Machine learning signal generator with proper OOS tracking.

    FIX #3 (v4.0): Now tracks and reports test AUC, not train AUC.
    FIX 1.1 (v4.2): Drops last training row to prevent label leakage at window boundaries.
    v4.3 ENHANCEMENT: Uses GradientBoosting/RandomForest for better predictive power.
    """

    def __init__(self, config: ScientificConfig):
        self.config = config
        self.model = None
        self.scaler = None

        # Track predictions and outcomes for calibration
        self.all_predictions = []
        self.all_outcomes = []
        self.feature_importances = {}

        # NEW: Track per-window test AUC
        self.window_test_aucs = []
        self.all_regimes = []  # For regime-specific analysis

    def _create_model(self):
        """
        v4.3: Create ML model based on config.ml_model_type.
        Options: 'logistic', 'gradient_boosting', 'random_forest'
        """
        model_type = getattr(self.config, 'ml_model_type', 'gradient_boosting')

        if model_type == 'gradient_boosting':
            try:
                from sklearn.ensemble import GradientBoostingClassifier
                return GradientBoostingClassifier(
                    n_estimators=getattr(self.config, 'ml_n_estimators', 100),
                    max_depth=getattr(self.config, 'ml_max_depth', 4),
                    learning_rate=getattr(self.config, 'ml_learning_rate', 0.05),
                    subsample=0.8,
                    min_samples_split=20,
                    min_samples_leaf=10,
                    random_state=42
                )
            except ImportError:
                pass

        elif model_type == 'random_forest':
            try:
                from sklearn.ensemble import RandomForestClassifier
                return RandomForestClassifier(
                    n_estimators=getattr(self.config, 'ml_n_estimators', 100),
                    max_depth=getattr(self.config, 'ml_max_depth', 4),
                    min_samples_split=20,
                    min_samples_leaf=10,
                    random_state=42,
                    n_jobs=-1
                )
            except ImportError:
                pass

        # Fallback to logistic regression
        from sklearn.linear_model import LogisticRegression
        return LogisticRegression(
            penalty='l1',
            solver='saga',
            C=1/self.config.ml_regularization,
            max_iter=1000,
            random_state=42
        )

    def fit_predict(self, train_df: pd.DataFrame, test_df: pd.DataFrame,
                    test_regimes: np.ndarray = None) -> Tuple[np.ndarray, float]:
        """
        Fit model and predict, tracking OOS performance.

        Returns: (predicted_probabilities, test_auc)

        FIX #3: Now returns TEST AUC, not train AUC.
        FIX 1.1 (v4.2): Drops last training row to prevent label leakage.
        v4.3: Uses ensemble models for better AUC.
        """
        try:
            from sklearn.preprocessing import StandardScaler
            from sklearn.metrics import roc_auc_score

            feature_cols = [c for c in self.config.ml_features if c in train_df.columns]
            if not feature_cols:
                return np.zeros(len(test_df)), 0.5

            X_train = train_df[feature_cols].fillna(0).values
            X_test = test_df[feature_cols].fillna(0).values

            # Labels: next day's return direction
            y_train = (train_df['ret_1'].shift(-1) > 0).astype(int).fillna(0).values
            y_test = (test_df['ret_1'].shift(-1) > 0).astype(int).fillna(0).values

            # FIX 1.1: Drop the last training row - its label uses the first test day's return
            if len(X_train) > 10:
                X_train = X_train[:-1]
                y_train = y_train[:-1]

            # Standardize using TRAINING statistics only
            self.scaler = StandardScaler()
            X_train_scaled = self.scaler.fit_transform(X_train)
            X_test_scaled = self.scaler.transform(X_test)

            # v4.3: Create model based on config
            self.model = self._create_model()
            self.model.fit(X_train_scaled, y_train)

            # Predict on test
            probs = self.model.predict_proba(X_test_scaled)[:, 1]

            # Store for calibration analysis (excluding last point which has no outcome)
            for i in range(len(probs) - 1):
                self.all_predictions.append(probs[i])
                self.all_outcomes.append(y_test[i])
                if test_regimes is not None and i < len(test_regimes):
                    self.all_regimes.append(test_regimes[i])

            # Store feature importances (handle different model types)
            if hasattr(self.model, 'feature_importances_'):
                # GradientBoosting, RandomForest
                importances = self.model.feature_importances_
            elif hasattr(self.model, 'coef_'):
                # LogisticRegression
                importances = np.abs(self.model.coef_[0])
            else:
                importances = np.ones(len(feature_cols)) / len(feature_cols)

            for i, col in enumerate(feature_cols):
                if col not in self.feature_importances:
                    self.feature_importances[col] = []
                self.feature_importances[col].append(importances[i])

            # FIX #3: Compute TEST AUC (not train AUC)
            # Exclude last point since we don't have its outcome yet
            valid_mask = np.arange(len(probs) - 1)
            if len(valid_mask) > 10 and len(np.unique(y_test[valid_mask])) > 1:
                test_auc = roc_auc_score(y_test[valid_mask], probs[valid_mask])
            else:
                test_auc = 0.5

            self.window_test_aucs.append(test_auc)

            return probs, test_auc

        except ImportError:
            return np.ones(len(test_df)) * 0.5, 0.5

    def get_calibration_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """Return all predictions and outcomes for calibration analysis."""
        return np.array(self.all_predictions), np.array(self.all_outcomes)

    def get_global_oos_auc(self) -> float:
        """Compute overall OOS AUC across all windows."""
        if len(self.all_predictions) < 10:
            return 0.5
        try:
            from sklearn.metrics import roc_auc_score
            preds = np.array(self.all_predictions)
            outcomes = np.array(self.all_outcomes)
            if len(np.unique(outcomes)) < 2:
                return 0.5
            return roc_auc_score(outcomes, preds)
        except:
            return 0.5

    def get_mean_window_auc(self) -> float:
        """Get mean test AUC across all windows."""
        if not self.window_test_aucs:
            return 0.5
        return np.mean(self.window_test_aucs)

    def get_average_window_auc(self) -> float:
        """Alias for get_mean_window_auc (backward compatibility)."""
        return self.get_mean_window_auc()

    def get_calibration_by_regime(self) -> Dict[int, Tuple[np.ndarray, np.ndarray]]:
        """Get predictions/outcomes separated by regime for analysis."""
        if not self.all_regimes:
            return {}

        result = {}
        preds = np.array(self.all_predictions)
        outcomes = np.array(self.all_outcomes)
        regimes = np.array(self.all_regimes[:len(preds)])

        for regime in np.unique(regimes):
            mask = regimes == regime
            result[regime] = (preds[mask], outcomes[mask])

        return result

    def reset_calibration_data(self):
        """Reset calibration tracking for new backtest."""
        self.all_predictions = []
        self.all_outcomes = []
        self.window_test_aucs = []
        self.all_regimes = []


# =============================================================================
# VOL PERCENTILE ENGINE (RENAMED FROM VRP - FIX #6)
# =============================================================================

class VolPercentileEngine:
    """
    Volatility Percentile calculator.

    FIX #6: Renamed from VRP since we don't have implied vol.
    This computes: current realized vol percentile vs historical distribution.
    """

    def __init__(self, config: ScientificConfig):
        self.config = config

    def calculate_vol_percentile(self, train_df: pd.DataFrame,
                                   test_df: pd.DataFrame) -> np.ndarray:
        """
        Calculate realized vol percentile using TRAINING distribution.

        Returns: array of percentiles [0, 1] for each test day.

        v4.1 OPTIMIZATION: Uses np.searchsorted for O(N*log(M)) instead of O(N*M).
        """
        vol_col = 'vol_20'
        if vol_col not in train_df.columns:
            return np.ones(len(test_df)) * 0.5

        # Use training period vol distribution
        historical_vol = train_df[vol_col].dropna().values

        if len(historical_vol) == 0:
            return np.ones(len(test_df)) * 0.5

        # Get test period vol
        test_vol = test_df[vol_col].fillna(method='ffill').values

        # OPTIMIZED: Use searchsorted for O(n*log(m)) instead of O(n*m)
        sorted_vol = np.sort(historical_vol)
        indices = np.searchsorted(sorted_vol, test_vol)
        percentiles = indices / len(sorted_vol)

        return percentiles


# =============================================================================
# CALIBRATION METRICS (Preserved from v3)
# =============================================================================

class CalibrationMetrics:
    """Calibration metrics for probability predictions (ISL p. 455)."""

    @staticmethod
    def compute_calibration_curve(predictions: np.ndarray, outcomes: np.ndarray,
                                   n_bins: int = 10) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Compute calibration curve data for reliability diagram."""
        if len(predictions) == 0 or len(outcomes) == 0:
            return np.array([]), np.array([]), np.array([])

        predictions = np.array(predictions)
        outcomes = np.array(outcomes)

        bin_edges = np.linspace(0, 1, n_bins + 1)
        bin_indices = np.digitize(predictions, bin_edges[1:-1])

        mean_predicted = []
        fraction_positive = []
        bin_counts = []

        for i in range(n_bins):
            mask = bin_indices == i
            if mask.sum() > 0:
                mean_predicted.append(predictions[mask].mean())
                fraction_positive.append(outcomes[mask].mean())
                bin_counts.append(mask.sum())
            else:
                mean_predicted.append(np.nan)
                fraction_positive.append(np.nan)
                bin_counts.append(0)

        return np.array(mean_predicted), np.array(fraction_positive), np.array(bin_counts)

    @staticmethod
    def expected_calibration_error(predictions: np.ndarray, outcomes: np.ndarray,
                                    n_bins: int = 10) -> float:
        """Compute Expected Calibration Error (ECE)."""
        if len(predictions) == 0:
            return 0.0

        mean_pred, frac_pos, counts = CalibrationMetrics.compute_calibration_curve(
            predictions, outcomes, n_bins
        )

        total = sum(counts)
        if total == 0:
            return 0.0

        ece = 0.0
        for i in range(len(counts)):
            if counts[i] > 0 and not np.isnan(mean_pred[i]) and not np.isnan(frac_pos[i]):
                weight = counts[i] / total
                ece += weight * abs(frac_pos[i] - mean_pred[i])

        return ece

    @staticmethod
    def maximum_calibration_error(predictions: np.ndarray, outcomes: np.ndarray,
                                   n_bins: int = 10) -> float:
        """Compute Maximum Calibration Error (MCE)."""
        if len(predictions) == 0:
            return 0.0

        mean_pred, frac_pos, counts = CalibrationMetrics.compute_calibration_curve(
            predictions, outcomes, n_bins
        )

        mce = 0.0
        for i in range(len(counts)):
            if counts[i] > 0 and not np.isnan(mean_pred[i]) and not np.isnan(frac_pos[i]):
                mce = max(mce, abs(frac_pos[i] - mean_pred[i]))

        return mce

    @staticmethod
    def brier_score(predictions: np.ndarray, outcomes: np.ndarray) -> float:
        """Compute Brier Score."""
        if len(predictions) == 0:
            return 0.0
        return np.mean((np.array(predictions) - np.array(outcomes)) ** 2)

    @staticmethod
    def brier_skill_score(predictions: np.ndarray, outcomes: np.ndarray) -> float:
        """Compute Brier Skill Score relative to climatology."""
        if len(predictions) == 0:
            return 0.0

        bs = CalibrationMetrics.brier_score(predictions, outcomes)
        base_rate = np.mean(outcomes)
        bs_ref = np.mean((base_rate - np.array(outcomes)) ** 2)

        if bs_ref == 0:
            return 0.0

        return 1 - (bs / bs_ref)


# =============================================================================
# SHARPE STATISTICS (Preserved from v3)
# =============================================================================

class SharpeStatistics:
    """Statistical tests for Sharpe ratio significance."""

    @staticmethod
    def sharpe_ratio(returns: np.ndarray, risk_free_rate: float = 0.0,
                     annualization: int = 252) -> float:
        """Calculate annualized Sharpe ratio."""
        if len(returns) == 0 or np.std(returns) == 0:
            return 0.0

        excess_returns = returns - risk_free_rate / annualization
        return np.mean(excess_returns) / np.std(excess_returns, ddof=1) * np.sqrt(annualization)

    @staticmethod
    def sharpe_standard_error(returns: np.ndarray, sharpe: float = None) -> float:
        """Calculate standard error of Sharpe ratio (Lo 2002)."""
        n = len(returns)
        if n < 2:
            return float('inf')

        if sharpe is None:
            sharpe = SharpeStatistics.sharpe_ratio(returns)

        sk = skew(returns) if len(returns) > 2 else 0
        ku = kurtosis(returns) if len(returns) > 3 else 0

        se_squared = (1 + 0.5 * sharpe**2 - sk * sharpe + (ku / 4) * sharpe**2) / (n - 1)

        return np.sqrt(max(se_squared, 1e-10))

    @staticmethod
    def sharpe_ttest(returns: np.ndarray, null_sharpe: float = 0.0,
                     risk_free_rate: float = 0.0, annualization: int = 252) -> Dict[str, float]:
        """Perform t-test for Sharpe ratio significance."""
        n = len(returns)
        if n < 3:
            return {
                'sharpe': 0.0, 't_stat': 0.0, 'p_value': 1.0,
                'se': float('inf'), 'df': 0, 'significant_5pct': False,
                'significant_1pct': False
            }

        sharpe = SharpeStatistics.sharpe_ratio(returns, risk_free_rate, annualization)
        se = SharpeStatistics.sharpe_standard_error(returns, sharpe)

        t_stat = (sharpe - null_sharpe) / se if se > 0 else 0
        df = n - 1
        p_value = 2 * (1 - t_dist.cdf(abs(t_stat), df))

        return {
            'sharpe': sharpe,
            't_stat': t_stat,
            'p_value': p_value,
            'se': se,
            'df': df,
            'significant_5pct': p_value < 0.05,
            'significant_1pct': p_value < 0.01,
            'ci_95_lower': sharpe - 1.96 * se,
            'ci_95_upper': sharpe + 1.96 * se
        }

    @staticmethod
    def sharpe_difference_ttest(returns1: np.ndarray, returns2: np.ndarray,
                                 risk_free_rate: float = 0.0,
                                 annualization: int = 252) -> Dict[str, float]:
        """Test if two Sharpe ratios are significantly different (Jobson-Korkie)."""
        n = min(len(returns1), len(returns2))
        if n < 5:
            return {
                'sharpe1': 0.0, 'sharpe2': 0.0, 'difference': 0.0,
                't_stat': 0.0, 'p_value': 1.0, 'significant_5pct': False
            }

        returns1 = returns1[:n]
        returns2 = returns2[:n]

        sharpe1 = SharpeStatistics.sharpe_ratio(returns1, risk_free_rate, annualization)
        sharpe2 = SharpeStatistics.sharpe_ratio(returns2, risk_free_rate, annualization)

        sig1, sig2 = np.std(returns1, ddof=1), np.std(returns2, ddof=1)

        if sig1 == 0 or sig2 == 0:
            return {
                'sharpe1': sharpe1, 'sharpe2': sharpe2, 'difference': sharpe1 - sharpe2,
                't_stat': 0.0, 'p_value': 1.0, 'significant_5pct': False
            }

        rho = np.corrcoef(returns1, returns2)[0, 1]

        theta = (1/n) * (2 * (1 - rho) + 0.5 * (sharpe1**2 + sharpe2**2 -
                 2 * sharpe1 * sharpe2 * rho**2))

        se_diff = np.sqrt(theta) if theta > 0 else 1e-8
        t_stat = (sharpe1 - sharpe2) / se_diff
        p_value = 2 * (1 - norm.cdf(abs(t_stat)))

        return {
            'sharpe1': sharpe1,
            'sharpe2': sharpe2,
            'difference': sharpe1 - sharpe2,
            't_stat': t_stat,
            'p_value': p_value,
            'se_diff': se_diff,
            'significant_5pct': p_value < 0.05,
            'significant_1pct': p_value < 0.01
        }

    @staticmethod
    def format_pvalue(p_value: float) -> str:
        """
        v4.7: Format p-value for display with scientific notation when very small.

        This fixes the issue where very small p-values display as 0.000000
        """
        if p_value == 0 or p_value < 1e-300:
            return "< 1e-300"  # Essentially zero
        elif p_value < 1e-100:
            return f"< 1e-100"
        elif p_value < 1e-50:
            return f"< 1e-50"
        elif p_value < 1e-10:
            return f"{p_value:.1e}"  # Scientific notation
        elif p_value < 0.001:
            return f"{p_value:.2e}"  # Scientific notation
        elif p_value < 0.01:
            return f"{p_value:.4f}"  # 4 decimal places
        else:
            return f"{p_value:.4f}"  # Standard format

    @staticmethod
    def bootstrap_sharpe_ci(returns: np.ndarray, n_bootstrap: int = 10000,
                            confidence: float = 0.95, risk_free_rate: float = 0.0,
                            annualization: int = 252) -> Dict[str, float]:
        """Bootstrap confidence interval for Sharpe ratio."""
        n = len(returns)
        if n < 5:
            return {
                'sharpe': 0.0, 'ci_lower': 0.0, 'ci_upper': 0.0, 'se_bootstrap': float('inf')
            }

        sharpe = SharpeStatistics.sharpe_ratio(returns, risk_free_rate, annualization)

        bootstrap_sharpes = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(returns, size=n, replace=True)
            boot_sharpe = SharpeStatistics.sharpe_ratio(sample, risk_free_rate, annualization)
            bootstrap_sharpes.append(boot_sharpe)

        bootstrap_sharpes = np.array(bootstrap_sharpes)

        alpha = 1 - confidence
        ci_lower = np.percentile(bootstrap_sharpes, 100 * alpha / 2)
        ci_upper = np.percentile(bootstrap_sharpes, 100 * (1 - alpha / 2))
        se_bootstrap = np.std(bootstrap_sharpes)

        return {
            'sharpe': sharpe,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'se_bootstrap': se_bootstrap,
            'bootstrap_mean': np.mean(bootstrap_sharpes),
            'bootstrap_median': np.median(bootstrap_sharpes)
        }


# =============================================================================
# STRATEGY ENGINE (FIXES #1, #2, #4, #5 - Complete rewrite of core logic)
# =============================================================================

class ScientificStrategyEngine:
    """
    Main strategy engine with all fixes applied.

    v4.4 CRITICAL FIX: PROPER OPTIONS PNL MODEL
    ────────────────────────────────────────────
    Previous versions used: pnl = position * underlying_return
    This is WRONG for options! Options premium selling profits from:
    - THETA: Time decay (collect premium daily)
    - VEGA: Vol contraction (profit when vol drops)
    - GAMMA: Penalty for big moves (squared)

    v4.4 uses: pnl = theta_pnl + vega_pnl + gamma_pnl + delta_pnl

    This properly models short premium strategies like strangles,
    iron condors, credit spreads.
    """

    def __init__(self, config: ScientificConfig):
        self.config = config
        self.loader = DataLoader(config)
        self.hmm = HMMRegimeDetector(config.hmm_n_states, config.hmm_n_iter)
        self.ml = MLSignalGenerator(config)
        self.vol_pctl = VolPercentileEngine(config)
        self.logger = Logger(config.logging)

        # v4.3: Track days since last rebalance
        self._days_since_rebalance = 0
        self._last_strategy = None

    def run(self, df: pd.DataFrame, ticker: str) -> pd.DataFrame:
        """Run strategy using walk-forward testing with Greeks-based PnL."""

        # Reset ML calibration tracking
        self.ml.reset_calibration_data()

        df = self.loader.add_features(df)

        # v4.3: Add smoothed volatility column for ART
        vol_window = getattr(self.config, 'vol_smoothing_window', 60)
        df['vol_smooth'] = df['ret_1'].rolling(vol_window, min_periods=20).std() * np.sqrt(252)

        # v4.4: Calculate vol changes for vega PnL
        df['vol_change'] = df['vol_smooth'].diff()

        if len(df) < self.config.min_train_size + self.config.test_window:
            return pd.DataFrame()

        results = []
        start_idx = self.config.train_window
        total_windows = (len(df) - start_idx) // self.config.test_window
        window_count = 0

        prev_strategy = None
        self._days_since_rebalance = 0
        self._last_strategy = 'Cash'
        global_day_counter = 0

        # v4.7 FIX: Running equity tracking for O(n) drawdown calculation
        running_equity = 1.0
        running_peak = 1.0

        while start_idx + self.config.test_window <= len(df):
            train_df = df.iloc[start_idx - self.config.train_window:start_idx]
            test_df = df.iloc[start_idx:start_idx + self.config.test_window]

            # =================================================================
            # FIX #1: HMM - Fit on train ONLY, predict on test
            # =================================================================
            self.hmm.fit(train_df)
            test_states, test_probs = self.hmm.predict(test_df)

            # =================================================================
            # ML - Fit on train, predict on test (v4.3: uses GradientBoosting)
            # =================================================================
            ml_probs, ml_test_auc = self.ml.fit_predict(train_df, test_df, test_states)

            # =================================================================
            # Vol Percentile - Uses training distribution
            # =================================================================
            vol_pctl = self.vol_pctl.calculate_vol_percentile(train_df, test_df)

            for i in range(len(test_df)):
                row = test_df.iloc[i]
                date = test_df.index[i]
                global_day_counter += 1

                regime_idx = test_states[i] if i < len(test_states) else 1
                regime = self.hmm.state_names[regime_idx] if regime_idx < len(self.hmm.state_names) else 'Neutral'
                ml_signal = ml_probs[i] if i < len(ml_probs) else 0.5
                vp = vol_pctl[i] if i < len(vol_pctl) else 0.5

                # v4.6 FIX 2.2: Use pre-computed atr_pct from DataFrame
                # (was: atr_14 / close which is already computed in add_features())
                atr_pct = row.get('atr_pct', 0.01)

                # =============================================================
                # v4.8 FIX: EX-ANTE RISK PARAMETERS (no look-ahead!)
                # These are based on PAST data known at decision time
                # =============================================================
                # Yesterday's return (current row's ret_1 is yesterday→today, already realized)
                recent_return = abs(row.get('ret_1', 0))
                # Vol-of-vol (uses past rolling window)
                vol_of_vol = row.get('vol_of_vol', 0.0)
                # Current vol for daily threshold
                current_vol_for_gate = row.get('vol_smooth', row.get('vol_20', 0.15))

                # =============================================================
                # v4.3: TURNOVER CONTROL - Only rebalance every N days
                # =============================================================
                rebalance_days = getattr(self.config, 'strategy_rebalance_days', 5)
                should_rebalance = (global_day_counter % rebalance_days == 0) or (self._last_strategy == 'Cash')

                if should_rebalance:
                    # v4.8: Use vol-focused strategy selection with ex-ante risk gates
                    strategy, base_position = self._select_strategy_v48(
                        regime, ml_signal, vp, atr_pct,
                        recent_return, vol_of_vol, current_vol_for_gate
                    )
                    self._last_strategy = strategy
                else:
                    strategy = self._last_strategy
                    _, base_position = self._select_strategy_v48(
                        regime, ml_signal, vp, atr_pct,
                        recent_return, vol_of_vol, current_vol_for_gate
                    )
                    if strategy == 'Cash':
                        base_position = 0.0

                # =============================================================
                # FIX #4: Apply strategy-specific leverage
                # =============================================================
                strategy_leverage = self.config.strategy_leverage.get_leverage(strategy)
                position = base_position * strategy_leverage

                # =============================================================
                # v4.3: CAP POSITION SIZE to reduce risk
                # =============================================================
                max_pos = getattr(self.config, 'max_position_size', 0.30)
                position = np.clip(position, -max_pos, max_pos)

                # =============================================================
                # v4.7 FIX: DRAWDOWN-BASED POSITION SCALING (O(n) optimized)
                # (was O(n²) - recalculated entire history every bar)
                # Now uses running_equity and running_peak
                # =============================================================
                if results:
                    # Update running equity with last bar's PnL
                    last_pnl = results[-1]['pnl']
                    running_equity = running_equity * (1 + last_pnl)
                    running_peak = max(running_peak, running_equity)
                    current_dd = (running_equity - running_peak) / running_peak if running_peak > 0 else 0.0

                    dd_limit_1 = getattr(self.config, 'dd_half_size_threshold', -0.10)
                    dd_limit_2 = getattr(self.config, 'dd_flat_threshold', -0.20)

                    if current_dd < dd_limit_2:
                        # Severe drawdown - go flat
                        position = 0.0
                        strategy = 'Cash'
                    elif current_dd < dd_limit_1:
                        # Moderate drawdown - half size
                        position *= 0.5

                # =============================================================
                # v4.4 CRITICAL: Use Greeks-based PnL model
                # =============================================================
                if i < len(test_df) - 1:
                    next_ret = test_df['ret_1'].iloc[i + 1]
                    next_vol_change = test_df['vol_change'].iloc[i + 1] if 'vol_change' in test_df.columns else 0
                else:
                    next_ret = 0
                    next_vol_change = 0

                # Clean vol change
                if np.isnan(next_vol_change):
                    next_vol_change = 0

                # v4.9: Calculate Greeks-based PnL with asset-specific scaling
                greeks_pnl = OptionsGreeksPnLModel.calculate_pnl(
                    strategy=strategy,
                    position_size=abs(position),
                    price_return=next_ret,
                    vol_change=next_vol_change,
                    days=1.0,
                    ticker=ticker  # v4.9: Asset-specific Greek scaling
                )

                raw_pnl = greeks_pnl['total_pnl']
                theta_pnl = greeks_pnl['theta_pnl']
                vega_pnl = greeks_pnl['vega_pnl']
                gamma_pnl = greeks_pnl['gamma_pnl']
                delta_pnl = greeks_pnl['delta_pnl']

                # =============================================================
                # Calculate current vol (for ART scaling)
                # v4.6 FIX 2.1: Use 'is not None' check instead of truthiness
                # =============================================================
                smooth_vol = test_df['vol_smooth'].iloc[i] if 'vol_smooth' in test_df.columns else None
                if smooth_vol is not None and not np.isnan(smooth_vol):
                    current_vol = smooth_vol
                elif 'vol_20' in test_df.columns:
                    current_vol = test_df['vol_20'].iloc[i]
                else:
                    current_vol = 0.15

                # =============================================================
                # v4.8 FIX: REMOVED LOOK-AHEAD GAMMA KILL-SWITCH
                #
                # PROBLEM: Previous versions used gamma_pnl (which depends on
                # next_ret) to decide to go to cash. This is LOOK-AHEAD BIAS!
                # We were seeing the future loss and then avoiding it.
                #
                # The ex-ante risk controls are already in _select_strategy_v44:
                # - Extreme vol cutoff (98th percentile → smaller position)
                # - Vol-of-vol check (high vol-of-vol → avoid)
                # - Recent large moves check (based on PAST returns)
                # =============================================================
                # NO gamma_killed logic here - we take the PnL as it comes

                # =============================================================
                # FIX 3 (v4.5): SOFTEN ART VOL TARGETING
                # When vol spikes → REDUCE positions, don't amplify losses
                # =============================================================
                ratio = self.config.target_vol / max(current_vol, 0.05)
                art_scalar = np.clip(ratio, 0.5, 1.2)
                scaled_pnl = raw_pnl * art_scalar

                # =============================================================
                # v4.7 FIX: Weekend theta/gamma PROPERLY WIRED
                # Now uses config.weekend_theta.get_weekend_factor()
                # and stores actual factor in weekend_adjustment
                # =============================================================
                day_of_week = date.dayofweek if hasattr(date, 'dayofweek') else 0
                weekend_adjustment = 1.0

                if day_of_week == 4:  # Friday
                    if strategy not in ['Cash']:
                        # v4.7: Actually use weekend_theta config
                        dte = self.config.dte_weekly if self.config.use_weekly else self.config.dte_monthly
                        weekend_factor = self.config.weekend_theta.get_weekend_factor(dte=dte)
                        weekend_adjustment = weekend_factor

                        weekend_days = 2.0  # Sat + Sun

                        # Add extra theta for weekend (positive = collect premium)
                        extra_theta = greeks_pnl['theta_pnl'] * weekend_days

                        # v4.6: Also add extra gamma penalty (50% of weekend days)
                        extra_gamma = greeks_pnl['gamma_pnl'] * weekend_days * 0.5

                        # Apply weekend factor to extra Greeks
                        scaled_pnl += (extra_theta + extra_gamma) * weekend_factor

                adjusted_pnl = scaled_pnl

                # =============================================================
                # Transaction costs - v4.6 FIX 2.3: Proportional to actual position change
                # (was: max(position_change, 1) which over-penalized small adjustments)
                # =============================================================
                tx_cost = 0
                if strategy != prev_strategy:
                    prev_pos = results[-1]['position'] if results else 0
                    position_change = abs(position - prev_pos)
                    # v4.6: Cost proportional to position_change with no forced minimum
                    tx_cost = (self.config.tx_commission + self.config.tx_spread +
                              self.config.tx_slippage) * position_change

                prev_strategy = strategy

                results.append({
                    'date': date,
                    'close': row['close'],
                    'ret_1': row['ret_1'],
                    'strategy': strategy,
                    'position': position,
                    'base_position': base_position,
                    'strategy_leverage': strategy_leverage,
                    'regime': regime,
                    'regime_idx': regime_idx,
                    'ml_prob': ml_signal,
                    'ml_test_auc': ml_test_auc,
                    'vol_percentile': vp,
                    'raw_pnl': raw_pnl,
                    'theta_pnl': theta_pnl,
                    'vega_pnl': vega_pnl,
                    'gamma_pnl': gamma_pnl,
                    'delta_pnl': delta_pnl,
                    'art_scalar': art_scalar,
                    'weekend_adjustment': weekend_adjustment,
                    'tx_cost': tx_cost,
                    'pnl': adjusted_pnl - tx_cost,
                    'regime_prob_0': test_probs[i, 0] if test_probs.shape[1] > 0 else 0,
                    'regime_prob_1': test_probs[i, 1] if test_probs.shape[1] > 1 else 0,
                    'regime_prob_2': test_probs[i, 2] if test_probs.shape[1] > 2 else 0,
                    'day_of_week': day_of_week,
                    'is_friday': int(day_of_week == 4),
                    'atr_pct': atr_pct,
                    'vol_smooth': current_vol,
                    'vol_change': next_vol_change
                })

            start_idx += self.config.test_window
            window_count += 1

            if self.config.logging.show_progress:
                self.logger.progress(window_count, total_windows, "Walk-forward")

        results_df = pd.DataFrame(results)
        if results_df.empty:
            return results_df

        results_df.set_index('date', inplace=True)
        results_df['equity'] = (1 + results_df['pnl']).cumprod() * self.config.account_size
        rolling_max = results_df['equity'].cummax()
        results_df['drawdown'] = (results_df['equity'] - rolling_max) / rolling_max

        # Turnover tracking
        results_df['position_change'] = results_df['position'].diff().abs().fillna(0)
        results_df['strategy_change'] = (results_df['strategy'] != results_df['strategy'].shift(1)).astype(int)

        # Forecast columns for analysis
        results_df['forecast'] = (results_df['ml_prob'] * 2 - 1) * results_df['ret_1'].rolling(20).std() * self.config.forecast_horizon
        results_df['realized_ret_h'] = results_df['close'].shift(-self.config.forecast_horizon) / results_df['close'] - 1

        return results_df

    def _select_strategy_v44(self, regime: str, ml_signal: float, vol_pctl: float,
                              atr_pct: float) -> Tuple[str, float]:
        """
        v4.6 ENHANCED: Strategy selection focused on VOL MEAN REVERSION.

        KEY INSIGHT: Don't try to predict direction (ML AUC ~0.50)!
        Instead, sell premium when vol is HIGH and collect theta + vega PnL
        as vol mean-reverts.

        v4.6 ENHANCEMENTS:
        - FIX 3.2a: Extreme vol cutoff (98th pctl → reduce/flat)
        - FIX 3.2b: ML confidence nudges position size (not direction)

        v4.5 RETAINED:
        - Vol threshold at 75th percentile
        - Focus on delta-neutral strategies
        """
        # v4.5: Use config threshold (default 0.75)
        vol_threshold = getattr(self.config, 'vol_percentile_threshold', 0.75)

        # Gate 1: Only trade when vol is elevated (premium is rich)
        if vol_pctl < vol_threshold:
            return 'Cash', 0.0

        # Gate 2: Skip extremely quiet markets (no movement = no premium)
        atr_threshold = getattr(self.config, 'atr_sideways_threshold', 0.002)
        if atr_pct < atr_threshold:
            return 'Cash', 0.0

        # =============================================================
        # v4.6 FIX 3.2a: EXTREME VOL CUTOFF
        # Above 98th percentile → either go smaller or flat
        # This cuts off worst gamma tails, improving Sortino
        # =============================================================
        extreme_vol_cutoff = getattr(self.config, 'extreme_vol_cutoff', 0.98)
        if vol_pctl >= extreme_vol_cutoff:
            # Too crazy - reduce to conservative position or go flat
            return 'IronCondor', 0.25  # Very conservative

        # Strategy selection based on regime
        # ALL strategies are premium-focused, NOT directional betting
        if regime == 'Bull':
            base_position = 0.5
            strategy = 'IronCondor'
        elif regime == 'Bear':
            base_position = 0.5
            strategy = 'IronCondor'
        else:
            base_position = 0.6
            strategy = 'Strangle'

        # =============================================================
        # v4.6 FIX 3.2b: ML CONFIDENCE SIZE ADJUSTMENT
        # Use ML confidence to nudge size (not direction!)
        # When ML is very confident, take slightly more exposure
        # Cap at +15% to avoid overfit
        # =============================================================
        ml_confidence = abs(ml_signal - 0.5)  # 0 = unsure, 0.5 = very confident
        size_multiplier = 1.0 + min(ml_confidence, 0.15)  # Max +15%
        base_position *= size_multiplier

        return strategy, base_position

    def _select_strategy_v48(self, regime: str, ml_signal: float, vol_pctl: float,
                              atr_pct: float, recent_return: float, vol_of_vol: float,
                              current_vol: float) -> Tuple[str, float]:
        """
        v4.8 FIXED: Strategy selection with EX-ANTE risk gates only.

        CRITICAL FIX: All risk checks use PAST/CURRENT data only!
        No look-ahead bias - we don't see future returns.

        EX-ANTE RISK GATES:
        1. Recent large move gate - if yesterday had big move, reduce exposure
        2. Vol-of-vol gate - if vol is unstable, reduce/avoid
        3. Extreme vol cutoff - if vol at 98th pctl, reduce
        4. ATR gate - if market too quiet, skip
        """
        # Gate 1: Only trade when vol is elevated (premium is rich)
        vol_threshold = getattr(self.config, 'vol_percentile_threshold', 0.75)
        if vol_pctl < vol_threshold:
            return 'Cash', 0.0

        # Gate 2: Skip extremely quiet markets
        atr_threshold = getattr(self.config, 'atr_sideways_threshold', 0.002)
        if atr_pct < atr_threshold:
            return 'Cash', 0.0

        # =============================================================
        # v4.8 EX-ANTE GATE 1: RECENT LARGE MOVE CHECK
        # If yesterday had a move > 2x daily vol, reduce exposure today
        # This is NOT look-ahead - we're using PAST realized return
        # =============================================================
        large_move_threshold = getattr(self.config, 'large_move_threshold', 2.0)
        daily_vol = current_vol / np.sqrt(252) if current_vol > 0 else 0.01

        recent_move_scalar = 1.0
        if recent_return > 3.0 * daily_vol:
            # Huge move yesterday → go flat today (too risky after big gap)
            return 'Cash', 0.0
        elif recent_return > large_move_threshold * daily_vol:
            # Big move yesterday → reduce exposure today
            recent_move_scalar = 0.5

        # =============================================================
        # v4.8 EX-ANTE GATE 2: VOL-OF-VOL CHECK
        # High vol-of-vol = vol is unstable → bad for premium selling
        # Only trade when vol is mean-reverting, not expanding
        # =============================================================
        vol_of_vol_threshold = getattr(self.config, 'vol_of_vol_threshold', 0.15)

        vov_scalar = 1.0
        if not np.isnan(vol_of_vol) and vol_of_vol > 0:
            if vol_of_vol > vol_of_vol_threshold * 1.5:
                # Vol very unstable - minimal exposure
                vov_scalar = 0.3
            elif vol_of_vol > vol_of_vol_threshold:
                # Vol is unstable - reduce exposure
                vov_scalar = 0.6

        # =============================================================
        # v4.8 EX-ANTE GATE 3: EXTREME VOL CUTOFF
        # Above 98th percentile → go much smaller
        # =============================================================
        extreme_vol_cutoff = getattr(self.config, 'extreme_vol_cutoff', 0.98)
        extreme_scalar = 1.0
        if vol_pctl >= extreme_vol_cutoff:
            extreme_scalar = 0.4

        # Strategy selection based on regime
        if regime == 'Bull':
            base_position = 0.5
            strategy = 'IronCondor'
        elif regime == 'Bear':
            base_position = 0.5
            strategy = 'IronCondor'
        else:
            base_position = 0.6
            strategy = 'Strangle'

        # ML confidence size adjustment (retained from v4.6)
        ml_confidence = abs(ml_signal - 0.5)
        size_multiplier = 1.0 + min(ml_confidence, 0.15)

        # Apply all ex-ante scalars
        base_position *= size_multiplier * recent_move_scalar * vov_scalar * extreme_scalar

        # If position is too small after all gates, go to cash
        if base_position < 0.15:
            return 'Cash', 0.0

        return strategy, base_position

    def _select_strategy_v43(self, regime: str, ml_signal: float, vol_pctl: float,
                              atr_pct: float) -> Tuple[str, float]:
        """
        v4.3 strategy selection (kept for comparison).
        Uses ML-based directional betting - now deprecated in v4.4.
        """
        vol_threshold = getattr(self.config, 'vol_percentile_threshold', 0.85)
        if vol_pctl < vol_threshold:
            return 'Cash', 0.0

        atr_threshold = getattr(self.config, 'atr_sideways_threshold', 0.005)
        is_sideways = atr_pct < atr_threshold

        min_prob = getattr(self.config, 'ml_min_probability', 0.55)
        strong_prob = min_prob + 0.10

        if regime == 'Bull':
            if ml_signal > strong_prob and not is_sideways:
                return 'BullPutSpread', 0.7
            elif ml_signal > min_prob:
                return 'IronCondor', 0.4
            else:
                return 'Cash', 0.0

        elif regime == 'Bear':
            if ml_signal < (1 - strong_prob) and not is_sideways:
                return 'BearCallSpread', -0.7
            elif ml_signal < (1 - min_prob):
                return 'IronCondor', -0.4
            else:
                return 'Cash', 0.0

        else:
            if is_sideways:
                return 'Cash', 0.0
            elif ml_signal > strong_prob:
                return 'ShortPut', 0.3
            elif ml_signal < (1 - strong_prob):
                return 'ShortCall', -0.3
            elif abs(ml_signal - 0.5) < 0.05:
                return 'Strangle', 0.2
            else:
                return 'Cash', 0.0

        return 'Cash', 0.0

    def _select_strategy(self, regime: str, ml_signal: float, vol_pctl: float) -> Tuple[str, float]:
        """Legacy strategy selection (kept for compatibility)."""
        if vol_pctl < self.config.vol_percentile_threshold:
            return 'Cash', 0.0

        if regime == 'Bull':
            if ml_signal > 0.6:
                return 'BullPutSpread', 1.0
            elif ml_signal > 0.4:
                return 'IronCondor', 0.5
            else:
                return 'Cash', 0.0
        elif regime == 'Bear':
            if ml_signal < 0.4:
                return 'BearCallSpread', -1.0
            elif ml_signal < 0.6:
                return 'IronCondor', -0.5
            else:
                return 'Cash', 0.0
        else:
            if ml_signal > 0.55:
                return 'ShortPut', 0.5
            elif ml_signal < 0.45:
                return 'ShortCall', -0.5
            else:
                return 'Strangle', 0.3

        return 'Cash', 0.0

    def get_ml_calibration_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """Get ML predictions and outcomes for calibration analysis."""
        return self.ml.get_calibration_data()

    def get_ml_global_oos_auc(self) -> float:
        """Get overall OOS AUC."""
        return self.ml.get_global_oos_auc()

    def get_ml_mean_window_auc(self) -> float:
        """Get mean per-window test AUC."""
        return self.ml.get_mean_window_auc()

    def get_turnover_metrics(self, results_df: pd.DataFrame = None) -> Dict[str, float]:
        """
        Calculate turnover metrics from results.

        v4.0 NEW: Track position changes and strategy switches.
        """
        if results_df is None:
            # Try to get from last run if stored
            return {
                'daily_turnover': 0.0,
                'annual_turnover': 0.0,
                'strategy_changes': 0,
                'avg_holding_period': 0.0
            }

        if 'position_change' not in results_df.columns:
            return {
                'daily_turnover': 0.0,
                'annual_turnover': 0.0,
                'strategy_changes': 0,
                'avg_holding_period': 0.0
            }

        daily_turnover = results_df['position_change'].mean()
        strategy_changes = results_df['strategy_change'].sum() if 'strategy_change' in results_df.columns else 0

        return {
            'daily_turnover': daily_turnover,
            'annual_turnover': daily_turnover * 252 * 100,
            'strategy_changes': int(strategy_changes),
            'avg_holding_period': len(results_df) / max(strategy_changes, 1)
        }


# =============================================================================
# METRICS CALCULATOR (Enhanced with new metrics)
# =============================================================================

class MetricsCalculator:
    """
    Calculate comprehensive performance metrics.

    VaR/ES SIGN CONVENTION NOTE:
    - VaR 95% is the 5th percentile of returns (a negative number for most strategies)
    - ES 95% is the mean of returns below VaR (also negative)
    - VaR Loss / ES Loss are the absolute values (positive loss numbers)

    Example: If VaR 95% = -0.023, that means "5% of days have returns worse than -2.3%"
             VaR Loss 95% = 0.023 (same info as positive loss)
    """

    @staticmethod
    def calculate_all(df: pd.DataFrame, ml_engine: MLSignalGenerator = None) -> Dict[str, float]:
        """Calculate all performance metrics."""
        metrics = {}
        if 'pnl' not in df.columns or len(df) == 0:
            return metrics

        returns = df['pnl']
        equity = df['equity'] if 'equity' in df.columns else (1 + returns).cumprod()

        # Basic returns
        total_return = (equity.iloc[-1] / equity.iloc[0]) - 1
        metrics['Total Return'] = total_return

        n_years = len(df) / 252
        metrics['CAGR'] = (equity.iloc[-1] / equity.iloc[0]) ** (1 / max(n_years, 0.01)) - 1 if equity.iloc[0] > 0 else 0

        # Risk metrics
        ann_vol = returns.std() * np.sqrt(252)
        metrics['Realized Vol'] = ann_vol
        metrics['Sharpe'] = (returns.mean() * 252) / ann_vol if ann_vol > 0 else 0

        downside = returns[returns < 0]
        downside_vol = downside.std() * np.sqrt(252) if len(downside) > 2 else ann_vol
        metrics['Sortino'] = (returns.mean() * 252) / downside_vol if downside_vol > 0 else 0

        rolling_max = equity.cummax()
        drawdown = (equity - rolling_max) / rolling_max
        metrics['Max Drawdown'] = drawdown.min()
        metrics['Calmar'] = metrics['CAGR'] / abs(metrics['Max Drawdown']) if metrics['Max Drawdown'] != 0 else 0

        winning_days = (returns > 0).sum()
        total_days = (returns != 0).sum()
        metrics['Win Rate'] = winning_days / total_days if total_days > 0 else 0

        # VaR/ES: Returns at tail (negative numbers for losses)
        # Sign convention: negative = loss, matches return sign
        metrics['VaR 95%'] = returns.quantile(0.05)
        tail_returns = returns[returns <= metrics['VaR 95%']]
        metrics['ES 95%'] = tail_returns.mean() if len(tail_returns) > 0 else metrics['VaR 95%']

        # VaR/ES Loss: Absolute values (positive numbers for losses)
        # Convention: many risk dashboards prefer positive loss numbers
        metrics['VaR Loss 95%'] = abs(metrics['VaR 95%'])
        metrics['ES Loss 95%'] = abs(metrics['ES 95%'])

        metrics['Skewness'] = returns.skew()
        metrics['Kurtosis'] = returns.kurtosis()

        gains = returns[returns > 0].sum()
        losses = -returns[returns < 0].sum()
        metrics['Omega'] = gains / losses if losses > 0 else float('inf')
        metrics['Profit Factor'] = gains / losses if losses > 0 else float('inf')

        # Transaction costs
        if 'tx_cost' in df.columns:
            metrics['Total TX Cost'] = df['tx_cost'].sum()
            metrics['Num Trades'] = df['strategy_change'].sum() if 'strategy_change' in df.columns else 0

        # Turnover metrics (NEW)
        if 'position_change' in df.columns:
            metrics['Daily Turnover %'] = df['position_change'].mean() * 100
            metrics['Annual Turnover %'] = df['position_change'].mean() * 252 * 100

        # ML metrics - FIX #3: Use OOS AUC
        if ml_engine is not None:
            # Handle both MLSignalGenerator and ScientificStrategyEngine
            if hasattr(ml_engine, 'ml'):
                # It's a ScientificStrategyEngine - use its ml attribute
                metrics['ML OOS AUC (Global)'] = ml_engine.ml.get_global_oos_auc()
                metrics['ML OOS AUC (Mean Window)'] = ml_engine.ml.get_mean_window_auc()
            elif hasattr(ml_engine, 'get_global_oos_auc'):
                # It's an MLSignalGenerator
                metrics['ML OOS AUC (Global)'] = ml_engine.get_global_oos_auc()
                metrics['ML OOS AUC (Mean Window)'] = ml_engine.get_mean_window_auc()
        elif 'ml_test_auc' in df.columns:
            metrics['ML OOS AUC (Last)'] = df['ml_test_auc'].iloc[-1]

        metrics['Vol Target Hit'] = 0.10 <= ann_vol <= 0.14

        # Weekend theta impact (NEW)
        if 'weekend_adjustment' in df.columns:
            friday_mask = df['is_friday'] == 1 if 'is_friday' in df.columns else pd.Series([False] * len(df))
            if friday_mask.any():
                metrics['Avg Weekend Theta Adj'] = df.loc[friday_mask, 'weekend_adjustment'].mean()

        return metrics


# =============================================================================
# NEW VISUALIZATION: ML LIFT CURVE / RANK BUCKET PLOT
# =============================================================================

def plot_ml_lift_curve(predictions: np.ndarray, outcomes: np.ndarray, returns: np.ndarray,
                        ticker: str = "", n_buckets: int = 10,
                        pdf: Optional[PdfPages] = None) -> Optional[plt.Figure]:
    """
    Plot ML lift curve - average return by prediction quintile.

    Shows whether higher ML probability leads to higher returns.
    """
    setup_dark_theme()

    if len(predictions) < 50:
        reset_light_theme()
        return None

    # Align arrays
    n = min(len(predictions), len(outcomes), len(returns))
    predictions = predictions[:n]
    outcomes = outcomes[:n]
    returns = returns[:n]

    # Create buckets based on predicted probability
    bucket_edges = np.percentile(predictions, np.linspace(0, 100, n_buckets + 1))
    bucket_edges[-1] += 0.001  # Ensure max is included
    bucket_indices = np.digitize(predictions, bucket_edges[1:])

    bucket_data = []
    for i in range(n_buckets):
        mask = bucket_indices == i
        if mask.sum() > 0:
            bucket_data.append({
                'bucket': i + 1,
                'mean_prob': predictions[mask].mean(),
                'mean_return': returns[mask].mean() * 10000,  # bps
                'hit_rate': outcomes[mask].mean(),
                'count': mask.sum()
            })

    bucket_df = pd.DataFrame(bucket_data)

    fig, axes = plt.subplots(1, 2, figsize=(16, 7), facecolor=DarkTheme.BACKGROUND)

    # Left: Return by bucket
    ax1 = axes[0]
    ax1.set_facecolor(DarkTheme.PANEL)

    colors = [DarkTheme.GREEN if r > 0 else DarkTheme.RED for r in bucket_df['mean_return']]
    ax1.bar(bucket_df['bucket'], bucket_df['mean_return'], color=colors,
            alpha=0.8, edgecolor='white')
    ax1.axhline(0, color=DarkTheme.TEXT_DIM, linestyle='--')
    ax1.set_xlabel('ML Probability Decile (1=Lowest, 10=Highest)', color=DarkTheme.TEXT)
    ax1.set_ylabel('Mean Next-Day Return (bps)', color=DarkTheme.TEXT)
    ax1.set_title(f'{ticker} – ML Lift Curve: Return by Prediction Bucket',
                  fontsize=14, color=DarkTheme.TEXT)
    ax1.grid(True, alpha=0.3, color=DarkTheme.GRID)

    # Right: Hit rate by bucket
    ax2 = axes[1]
    ax2.set_facecolor(DarkTheme.PANEL)

    ax2.bar(bucket_df['bucket'], bucket_df['hit_rate'] * 100,
            color=DarkTheme.BLUE, alpha=0.8, edgecolor='white')
    ax2.axhline(50, color=DarkTheme.GOLD, linestyle='--', label='Random (50%)')
    ax2.set_xlabel('ML Probability Decile', color=DarkTheme.TEXT)
    ax2.set_ylabel('Actual Up-Day Rate (%)', color=DarkTheme.TEXT)
    ax2.set_title(f'{ticker} – Hit Rate by Prediction Bucket',
                  fontsize=14, color=DarkTheme.TEXT)
    ax2.legend(loc='upper left', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax2.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.tight_layout()

    if pdf:
        pdf.savefig(fig, facecolor=fig.get_facecolor())
        plt.close(fig)
        reset_light_theme()
        return None

    reset_light_theme()
    return fig


# =============================================================================
# NEW VISUALIZATION: TURNOVER & EXPOSURE PLOTS
# =============================================================================

def plot_turnover_exposure(df: pd.DataFrame, ticker: str = "",
                            pdf: Optional[PdfPages] = None) -> Optional[plt.Figure]:
    """Plot turnover and position exposure time series."""
    setup_dark_theme()

    if len(df) < 20:
        reset_light_theme()
        return None

    fig, axes = plt.subplots(2, 2, figsize=(16, 10), facecolor=DarkTheme.BACKGROUND)

    # 1. Position exposure over time
    ax1 = axes[0, 0]
    ax1.set_facecolor(DarkTheme.PANEL)

    if 'position' in df.columns:
        ax1.fill_between(df.index, 0, df['position'], where=df['position'] > 0,
                        color=DarkTheme.GREEN, alpha=0.5, label='Long')
        ax1.fill_between(df.index, 0, df['position'], where=df['position'] < 0,
                        color=DarkTheme.RED, alpha=0.5, label='Short')
        ax1.plot(df.index, df['position'], color=DarkTheme.BLUE, linewidth=1)

    ax1.axhline(0, color=DarkTheme.TEXT_DIM, linestyle='--')
    ax1.set_title('Net Position Exposure Over Time', fontsize=12, color=DarkTheme.TEXT)
    ax1.set_ylabel('Position', color=DarkTheme.TEXT)
    ax1.legend(loc='upper left', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax1.grid(True, alpha=0.3, color=DarkTheme.GRID)

    # 2. Gross exposure (absolute position)
    ax2 = axes[0, 1]
    ax2.set_facecolor(DarkTheme.PANEL)

    if 'position' in df.columns:
        gross_exp = df['position'].abs()
        ax2.fill_between(df.index, 0, gross_exp, color=DarkTheme.ORANGE, alpha=0.5)
        ax2.plot(df.index, gross_exp, color=DarkTheme.ORANGE, linewidth=1)

    ax2.set_title('Gross Exposure Over Time', fontsize=12, color=DarkTheme.TEXT)
    ax2.set_ylabel('|Position|', color=DarkTheme.TEXT)
    ax2.grid(True, alpha=0.3, color=DarkTheme.GRID)

    # 3. Daily turnover
    ax3 = axes[1, 0]
    ax3.set_facecolor(DarkTheme.PANEL)

    if 'position_change' in df.columns:
        turnover = df['position_change'] * 100
        ax3.bar(df.index, turnover, color=DarkTheme.CYAN, alpha=0.7, width=1)
        ax3.axhline(turnover.mean(), color=DarkTheme.GOLD, linestyle='--',
                   label=f'Mean: {turnover.mean():.1f}%')

    ax3.set_title('Daily Turnover', fontsize=12, color=DarkTheme.TEXT)
    ax3.set_ylabel('Turnover (%)', color=DarkTheme.TEXT)
    ax3.legend(loc='upper right', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax3.grid(True, alpha=0.3, color=DarkTheme.GRID)

    # 4. Strategy allocation over time (stacked)
    ax4 = axes[1, 1]
    ax4.set_facecolor(DarkTheme.PANEL)

    if 'strategy' in df.columns:
        strategies = df['strategy'].unique()
        strategy_colors = {
            'Cash': DarkTheme.TEXT_DIM,
            'BullPutSpread': DarkTheme.GREEN,
            'BearCallSpread': DarkTheme.RED,
            'IronCondor': DarkTheme.PURPLE,
            'ShortPut': DarkTheme.CYAN,
            'ShortCall': DarkTheme.ORANGE,
            'Strangle': DarkTheme.PINK
        }

        # Create running count for each strategy
        for strat in strategies:
            if strat in strategy_colors:
                mask = df['strategy'] == strat
                ax4.fill_between(df.index, 0, mask.astype(int) * 0.5,
                               alpha=0.5, color=strategy_colors[strat], label=strat)

    ax4.set_title('Strategy Allocation Over Time', fontsize=12, color=DarkTheme.TEXT)
    ax4.set_ylabel('Active', color=DarkTheme.TEXT)
    ax4.legend(loc='upper left', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT, fontsize=8, ncol=2)
    ax4.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.suptitle(f'{ticker} – Turnover & Exposure Analysis', fontsize=14,
                fontweight='bold', color=DarkTheme.TEXT)
    plt.tight_layout(rect=[0, 0, 1, 0.96])

    if pdf:
        pdf.savefig(fig, facecolor=fig.get_facecolor())
        plt.close(fig)
        reset_light_theme()
        return None

    reset_light_theme()
    return fig


# =============================================================================
# NEW VISUALIZATION: CONFUSION MATRIX BY REGIME
# =============================================================================

def plot_confusion_by_regime(df: pd.DataFrame, predictions: np.ndarray,
                              outcomes: np.ndarray, ticker: str = "",
                              pdf: Optional[PdfPages] = None) -> Optional[plt.Figure]:
    """Plot confusion matrix analysis by regime."""
    setup_dark_theme()

    if len(predictions) < 50 or 'regime' not in df.columns:
        reset_light_theme()
        return None

    # Align
    n = min(len(predictions), len(outcomes), len(df))
    predictions = predictions[:n]
    outcomes = outcomes[:n]
    regimes = df['regime'].iloc[:n].values

    fig, axes = plt.subplots(1, 3, figsize=(18, 6), facecolor=DarkTheme.BACKGROUND)

    regime_names = ['Bull', 'Neutral', 'Bear']

    for idx, regime_name in enumerate(regime_names):
        ax = axes[idx]
        ax.set_facecolor(DarkTheme.PANEL)

        mask = regimes == regime_name
        if mask.sum() < 10:
            ax.text(0.5, 0.5, f'Insufficient data\n({mask.sum()} samples)',
                   ha='center', va='center', fontsize=12, color=DarkTheme.TEXT)
            ax.set_title(f'{regime_name} Regime', fontsize=12, color=DarkTheme.TEXT)
            ax.axis('off')
            continue

        regime_preds = predictions[mask]
        regime_outcomes = outcomes[mask]

        # Discretize predictions at 0.5
        pred_up = regime_preds > 0.5
        actual_up = regime_outcomes == 1

        # Confusion matrix
        TP = (pred_up & actual_up).sum()
        TN = (~pred_up & ~actual_up).sum()
        FP = (pred_up & ~actual_up).sum()
        FN = (~pred_up & actual_up).sum()

        cm = np.array([[TN, FP], [FN, TP]])

        # Plot as heatmap
        im = ax.imshow(cm, cmap='Blues', aspect='auto')

        # Add text annotations
        for i in range(2):
            for j in range(2):
                color = 'white' if cm[i, j] > cm.max() / 2 else DarkTheme.TEXT
                ax.text(j, i, f'{cm[i, j]}\n({cm[i, j] / cm.sum() * 100:.1f}%)',
                       ha='center', va='center', fontsize=11, color=color)

        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])
        ax.set_xticklabels(['Pred Down', 'Pred Up'], color=DarkTheme.TEXT)
        ax.set_yticklabels(['Actual Down', 'Actual Up'], color=DarkTheme.TEXT)

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0

        ax.set_title(f'{regime_name} Regime (n={mask.sum()})\nAcc: {accuracy:.1%} | Prec: {precision:.1%} | Rec: {recall:.1%}',
                    fontsize=12, color=DarkTheme.TEXT)

    plt.suptitle(f'{ticker} – ML Confusion Matrix by Regime', fontsize=14,
                fontweight='bold', color=DarkTheme.TEXT)
    plt.tight_layout(rect=[0, 0, 1, 0.94])

    if pdf:
        pdf.savefig(fig, facecolor=fig.get_facecolor())
        plt.close(fig)
        reset_light_theme()
        return None

    reset_light_theme()
    return fig


# =============================================================================
# PRESERVED VISUALIZATIONS FROM v3 (reliability diagram, sharpe significance)
# =============================================================================

def plot_reliability_diagram(predictions: np.ndarray, outcomes: np.ndarray,
                              n_bins: int = 10, ticker: str = "",
                              pdf: Optional[PdfPages] = None) -> Optional[plt.Figure]:
    """Plot reliability diagram (calibration curve) as in ISL p. 455."""
    setup_dark_theme()

    if len(predictions) < 50:
        reset_light_theme()
        return None

    mean_pred, frac_pos, counts = CalibrationMetrics.compute_calibration_curve(
        predictions, outcomes, n_bins
    )

    ece = CalibrationMetrics.expected_calibration_error(predictions, outcomes, n_bins)
    mce = CalibrationMetrics.maximum_calibration_error(predictions, outcomes, n_bins)
    brier = CalibrationMetrics.brier_score(predictions, outcomes)
    bss = CalibrationMetrics.brier_skill_score(predictions, outcomes)

    fig, axes = plt.subplots(1, 2, figsize=(16, 7), facecolor=DarkTheme.BACKGROUND)

    # Left: Reliability Diagram
    ax1 = axes[0]
    ax1.set_facecolor(DarkTheme.PANEL)

    ax1.plot([0, 1], [0, 1], '--', color=DarkTheme.TEXT_DIM, linewidth=2,
             label='Perfect Calibration', alpha=0.8)

    valid_mask = ~np.isnan(mean_pred) & ~np.isnan(frac_pos)
    if valid_mask.sum() > 0:
        x = mean_pred[valid_mask]
        y = frac_pos[valid_mask]
        n = counts[valid_mask]

        sizes = (n / n.max()) * 300 + 50
        ax1.scatter(x, y, s=sizes, c=DarkTheme.BLUE, alpha=0.8,
                   edgecolors='white', linewidth=1, zorder=5)

        sorted_idx = np.argsort(x)
        ax1.plot(x[sorted_idx], y[sorted_idx], '-', color=DarkTheme.BLUE,
                linewidth=2, alpha=0.8, label='Model Calibration')

    ax1.set_xlim([0, 1])
    ax1.set_ylim([0, 1])
    ax1.set_xlabel('Mean Predicted Probability', color=DarkTheme.TEXT, fontsize=12)
    ax1.set_ylabel('Fraction of Positives', color=DarkTheme.TEXT, fontsize=12)
    ax1.set_title(f'{ticker} – Reliability Diagram (ISL Fig 4.18)',
                  fontsize=14, color=DarkTheme.TEXT, fontweight='bold')
    ax1.legend(loc='upper left', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax1.grid(True, alpha=0.3, color=DarkTheme.GRID)

    metrics_text = (f'ECE: {ece:.4f}\n'
                   f'MCE: {mce:.4f}\n'
                   f'Brier: {brier:.4f}\n'
                   f'BSS: {bss:.4f}')
    ax1.text(0.95, 0.05, metrics_text, transform=ax1.transAxes, fontsize=11,
             verticalalignment='bottom', horizontalalignment='right',
             color=DarkTheme.TEXT,
             bbox=dict(boxstyle='round', facecolor=DarkTheme.PANEL,
                      edgecolor=DarkTheme.GRID, alpha=0.9))

    # Right: Histogram
    ax2 = axes[1]
    ax2.set_facecolor(DarkTheme.PANEL)

    pred_pos = predictions[outcomes == 1]
    pred_neg = predictions[outcomes == 0]

    bins = np.linspace(0, 1, 21)
    ax2.hist(pred_neg, bins=bins, alpha=0.7, color=DarkTheme.RED,
             label=f'Outcome=0 (n={len(pred_neg)})', edgecolor='white', linewidth=0.5)
    ax2.hist(pred_pos, bins=bins, alpha=0.7, color=DarkTheme.GREEN,
             label=f'Outcome=1 (n={len(pred_pos)})', edgecolor='white', linewidth=0.5)

    ax2.set_xlabel('Predicted Probability', color=DarkTheme.TEXT, fontsize=12)
    ax2.set_ylabel('Count', color=DarkTheme.TEXT, fontsize=12)
    ax2.set_title(f'{ticker} – Prediction Distribution by Outcome',
                  fontsize=14, color=DarkTheme.TEXT, fontweight='bold')
    ax2.legend(loc='upper center', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax2.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.tight_layout()

    if pdf:
        pdf.savefig(fig, facecolor=fig.get_facecolor())
        plt.close(fig)
        reset_light_theme()
        return None

    reset_light_theme()
    return fig


def plot_sharpe_significance(returns: np.ndarray, ticker: str = "",
                              pdf: Optional[PdfPages] = None,
                              precomputed_ttest: Dict = None,
                              precomputed_bootstrap: Dict = None,
                              n_bootstrap: int = 2000) -> Optional[plt.Figure]:
    """
    Plot Sharpe ratio with statistical significance analysis.

    FIX 2.5: Now accepts precomputed results to avoid double bootstrapping.
    Reduced default n_bootstrap from 5000 to 2000 for speed.

    Args:
        returns: Array of returns
        ticker: Asset ticker for title
        pdf: Optional PDF to save to
        precomputed_ttest: Pre-computed t-test results (optional)
        precomputed_bootstrap: Pre-computed bootstrap results (optional)
        n_bootstrap: Number of bootstrap samples if computing fresh
    """
    setup_dark_theme()

    if len(returns) < 30:
        reset_light_theme()
        return None

    # Use precomputed results if available, else compute
    if precomputed_ttest is not None:
        ttest_results = precomputed_ttest
    else:
        ttest_results = SharpeStatistics.sharpe_ttest(returns)

    if precomputed_bootstrap is not None:
        bootstrap_results = precomputed_bootstrap
        # Still need the distribution for plotting - but with fewer samples
        n = len(returns)
        bootstrap_sharpes = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(returns, size=n, replace=True)
            boot_sharpe = SharpeStatistics.sharpe_ratio(sample)
            bootstrap_sharpes.append(boot_sharpe)
        bootstrap_sharpes = np.array(bootstrap_sharpes)
    else:
        # Compute bootstrap with distribution in one pass
        n = len(returns)
        bootstrap_sharpes = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(returns, size=n, replace=True)
            boot_sharpe = SharpeStatistics.sharpe_ratio(sample)
            bootstrap_sharpes.append(boot_sharpe)
        bootstrap_sharpes = np.array(bootstrap_sharpes)

        # Derive CI and stats from distribution
        bootstrap_results = {
            'sharpe': SharpeStatistics.sharpe_ratio(returns),
            'ci_lower': np.percentile(bootstrap_sharpes, 2.5),
            'ci_upper': np.percentile(bootstrap_sharpes, 97.5),
            'se_bootstrap': np.std(bootstrap_sharpes),
            'bootstrap_mean': np.mean(bootstrap_sharpes),
            'bootstrap_median': np.median(bootstrap_sharpes)
        }

    fig, axes = plt.subplots(1, 2, figsize=(16, 7), facecolor=DarkTheme.BACKGROUND)

    # Left: Bootstrap distribution
    ax1 = axes[0]
    ax1.set_facecolor(DarkTheme.PANEL)

    ax1.hist(bootstrap_sharpes, bins=50, density=True, alpha=0.7,
             color=DarkTheme.HISTOGRAM, edgecolor='white', linewidth=0.5)

    sharpe = ttest_results['sharpe']
    ci_lower = bootstrap_results['ci_lower']
    ci_upper = bootstrap_results['ci_upper']

    ax1.axvline(sharpe, color=DarkTheme.BLUE, linewidth=3,
                label=f'Sharpe = {sharpe:.4f}')
    ax1.axvline(0, color=DarkTheme.RED, linewidth=2, linestyle='--',
                label='H₀: Sharpe = 0')
    ax1.axvline(ci_lower, color=DarkTheme.GREEN, linewidth=2, linestyle=':',
                label=f'95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]')
    ax1.axvline(ci_upper, color=DarkTheme.GREEN, linewidth=2, linestyle=':')

    ax1.axvspan(ci_lower, ci_upper, alpha=0.2, color=DarkTheme.GREEN)

    ax1.set_xlabel('Sharpe Ratio', color=DarkTheme.TEXT, fontsize=12)
    ax1.set_ylabel('Density', color=DarkTheme.TEXT, fontsize=12)
    ax1.set_title(f'{ticker} – Bootstrap Distribution of Sharpe Ratio',
                  fontsize=14, color=DarkTheme.TEXT, fontweight='bold')
    ax1.legend(loc='upper right', facecolor=DarkTheme.PANEL,
               edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT, fontsize=10)
    ax1.grid(True, alpha=0.3, color=DarkTheme.GRID)

    # Right: Summary
    ax2 = axes[1]
    ax2.set_facecolor(DarkTheme.PANEL)
    ax2.axis('off')

    sig_5 = "✓ Yes" if ttest_results['significant_5pct'] else "✗ No"
    sig_1 = "✓ Yes" if ttest_results['significant_1pct'] else "✗ No"

    summary_text = f"""
    SHARPE RATIO STATISTICAL SIGNIFICANCE TEST
    {'─' * 50}

    Sample Size (n):           {len(returns):,} days

    Point Estimate:
      Sharpe Ratio:            {sharpe:.4f}
      Standard Error:          {ttest_results['se']:.4f}

    T-Test (H₀: Sharpe = 0):
      t-statistic:             {ttest_results['t_stat']:.4f}
      p-value:                 {ttest_results['p_value']:.6f}
      Degrees of freedom:      {ttest_results['df']}

    Significance:
      At α = 0.05:             {sig_5}
      At α = 0.01:             {sig_1}

    Confidence Intervals:
      Analytical 95% CI:       [{ttest_results['ci_95_lower']:.4f}, {ttest_results['ci_95_upper']:.4f}]
      Bootstrap 95% CI:        [{ci_lower:.4f}, {ci_upper:.4f}]

    Bootstrap Statistics:
      Mean:                    {bootstrap_results['bootstrap_mean']:.4f}
      Median:                  {bootstrap_results['bootstrap_median']:.4f}
      Std Error:               {bootstrap_results['se_bootstrap']:.4f}
    """

    ax2.text(0.05, 0.95, summary_text, transform=ax2.transAxes, fontsize=11,
             verticalalignment='top', fontfamily='monospace',
             color=DarkTheme.TEXT,
             bbox=dict(boxstyle='round', facecolor=DarkTheme.PANEL,
                      edgecolor=DarkTheme.GRID, alpha=0.9))

    ax2.set_title(f'{ticker} – Sharpe Ratio T-Test Results',
                  fontsize=14, color=DarkTheme.TEXT, fontweight='bold', pad=20)

    plt.tight_layout()

    if pdf:
        pdf.savefig(fig, facecolor=fig.get_facecolor())
        plt.close(fig)
        reset_light_theme()
        return None

    reset_light_theme()
    return fig


# =============================================================================
# ADDITIONAL STANDARD PLOTS (Preserved from v3)
# =============================================================================

def plot_monthly_returns_line(ticker: str, df: pd.DataFrame, pdf: PdfPages):
    """Monthly returns as LINE plot."""
    setup_dark_theme()

    monthly_strategy = df['pnl'].resample('M').sum() * 100
    monthly_buyhold = df['ret_1'].resample('M').sum() * 100

    fig, ax = plt.subplots(figsize=(16, 6), facecolor=DarkTheme.BACKGROUND)
    ax.set_facecolor(DarkTheme.PANEL)

    ax.plot(monthly_strategy.index, monthly_strategy.values,
            color=DarkTheme.BLUE, linewidth=2, marker='o', markersize=4, label='Strategy')
    ax.plot(monthly_buyhold.index, monthly_buyhold.values,
            color=DarkTheme.ORANGE, linewidth=2, marker='s', markersize=4, label='Buy & Hold')

    ax.axhline(0, color=DarkTheme.TEXT_DIM, linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'{ticker} – Monthly Returns: Strategy vs Buy & Hold',
                fontsize=14, color=DarkTheme.TEXT)
    ax.set_xlabel('Date', color=DarkTheme.TEXT)
    ax.set_ylabel('Monthly Return (%)', color=DarkTheme.TEXT)
    ax.legend(loc='upper left', facecolor=DarkTheme.PANEL,
             edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.tight_layout()
    pdf.savefig(fig, facecolor=fig.get_facecolor())
    plt.close(fig)
    reset_light_theme()


def plot_price_with_trade_entries(ticker: str, df: pd.DataFrame, pdf: PdfPages):
    """Price with trades using ACTUAL prices."""
    setup_dark_theme()

    fig, ax = plt.subplots(figsize=(16, 6), facecolor=DarkTheme.BACKGROUND)
    ax.set_facecolor(DarkTheme.PANEL)

    prices = df['close']
    ax.plot(df.index, prices, color=DarkTheme.BLUE, linewidth=1.5, label='Price')

    if 'position' in df.columns:
        pos_change = df['position'].diff().fillna(0)
        long_mask = pos_change > 0
        short_mask = pos_change < 0

        ax.scatter(df.index[long_mask], prices[long_mask], marker='^', s=100,
                   c=DarkTheme.GREEN, label=f'Long Entry ({long_mask.sum()})',
                   zorder=5, edgecolors='white', linewidth=0.5)
        ax.scatter(df.index[short_mask], prices[short_mask], marker='v', s=100,
                   c=DarkTheme.ORANGE, label=f'Short Entry ({short_mask.sum()})',
                   zorder=5, edgecolors='white', linewidth=0.5)

    ax.set_title(f'{ticker} – Price with Trade Entries', fontsize=14, color=DarkTheme.TEXT)
    ax.set_xlabel('Date', color=DarkTheme.TEXT)
    ax.set_ylabel('Price', color=DarkTheme.TEXT)
    ax.legend(loc='upper left', facecolor=DarkTheme.PANEL,
             edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.tight_layout()
    pdf.savefig(fig, facecolor=fig.get_facecolor())
    plt.close(fig)
    reset_light_theme()


def plot_strategy_vs_buyhold(ticker: str, df: pd.DataFrame, pdf: PdfPages):
    """Strategy vs Buy & Hold with BOTH lines visible."""
    setup_dark_theme()

    fig, ax = plt.subplots(figsize=(16, 7), facecolor=DarkTheme.BACKGROUND)
    ax.set_facecolor(DarkTheme.PANEL)

    strategy_norm = df['equity'] / df['equity'].iloc[0]
    buyhold_norm = (1 + df['ret_1']).cumprod()

    ax.plot(df.index, strategy_norm, color=DarkTheme.BLUE, linewidth=2,
            label='Strategy', zorder=3)
    ax.plot(df.index, buyhold_norm, color=DarkTheme.ORANGE, linewidth=2,
            linestyle='--', label='Buy & Hold', zorder=2)

    ax.fill_between(df.index, strategy_norm, buyhold_norm,
                    where=(strategy_norm > buyhold_norm),
                    color=DarkTheme.GREEN, alpha=0.2, label='Outperformance')
    ax.fill_between(df.index, strategy_norm, buyhold_norm,
                    where=(strategy_norm <= buyhold_norm),
                    color=DarkTheme.RED, alpha=0.2, label='Underperformance')

    ax.axhline(1.0, color=DarkTheme.TEXT_DIM, linewidth=1, linestyle='--', alpha=0.5)

    ax.set_title(f'{ticker} – Strategy vs Buy & Hold (Walk-Forward)',
                fontsize=14, color=DarkTheme.TEXT)
    ax.set_xlabel('Date', color=DarkTheme.TEXT)
    ax.set_ylabel('Equity (indexed at 1.0)', color=DarkTheme.TEXT)
    ax.legend(loc='upper left', facecolor=DarkTheme.PANEL,
             edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.tight_layout()
    pdf.savefig(fig, facecolor=fig.get_facecolor())
    plt.close(fig)
    reset_light_theme()


def plot_cross_asset_comparison(all_results: Dict[str, pd.DataFrame], pdf: PdfPages):
    """Cross-asset strategy comparison."""
    setup_dark_theme()

    fig, ax = plt.subplots(figsize=(16, 8), facecolor=DarkTheme.BACKGROUND)
    ax.set_facecolor(DarkTheme.PANEL)

    colors = [DarkTheme.BLUE, DarkTheme.ORANGE, DarkTheme.GREEN,
              DarkTheme.RED, DarkTheme.PURPLE, DarkTheme.CYAN, DarkTheme.PINK]

    for i, (name, res_df) in enumerate(all_results.items()):
        if res_df.empty or 'equity' not in res_df.columns:
            continue
        normalized = res_df['equity'] / res_df['equity'].iloc[0]
        ax.plot(res_df.index, normalized, color=colors[i % len(colors)],
                linewidth=2, label=name)

    ax.axhline(1.0, color=DarkTheme.TEXT_DIM, linewidth=1, linestyle='--', alpha=0.5)
    ax.set_title('Strategy Equity Curves – Cross Asset Comparison',
                fontsize=14, color=DarkTheme.TEXT)
    ax.set_xlabel('Date', color=DarkTheme.TEXT)
    ax.set_ylabel('Equity (normalized)', color=DarkTheme.TEXT)
    ax.legend(loc='upper left', facecolor=DarkTheme.PANEL,
             edgecolor=DarkTheme.GRID, labelcolor=DarkTheme.TEXT)
    ax.grid(True, alpha=0.3, color=DarkTheme.GRID)

    plt.tight_layout()
    pdf.savefig(fig, facecolor=fig.get_facecolor())
    plt.close(fig)
    reset_light_theme()


# =============================================================================
# ENHANCED WEEKEND THETA TESTER (Now tests actually connected logic)
# =============================================================================

class WeekendThetaTester:
    """
    Weekend theta factor tester with statistical significance.

    NOW ACTUALLY TESTS THE WIRED WEEKEND THETA LOGIC (FIX #2).
    v4.1: Uses deepcopy of caller's config to preserve all settings.
    """

    def __init__(self, config: ScientificConfig):
        self.config = config
        self.results = {}
        self.returns_by_factor = {}
        self.logger = Logger(config.logging)

    def test_factors(self, df: pd.DataFrame, ticker: str,
                     factors: List[float]) -> Dict[float, Dict[str, float]]:
        """Test multiple weekend theta factors."""
        import copy

        self.logger.header("WEEKEND THETA FACTOR TESTING (v4 - NOW WIRED)")
        self.logger.info(f"Testing {len(factors)} factors: {factors}")

        results = {}
        returns_by_factor = {}

        for i, factor in enumerate(factors):
            self.logger.info(f"\n[{i+1}/{len(factors)}] Testing factor = {factor:.1f}")

            # FIX 2.3: Use deepcopy of caller's config to preserve all settings
            # Only override weekend theta parameters
            test_config = copy.deepcopy(self.config)
            test_config.logging.level = LogLevel.QUIET
            test_config.logging.show_progress = False
            test_config.weekend_theta.weekend_factor = factor
            test_config.weekend_theta.dte_7_factor = factor * 1.17
            test_config.weekend_theta.dte_14_factor = factor
            test_config.weekend_theta.dte_30_factor = factor * 0.83

            try:
                engine = ScientificStrategyEngine(test_config)
                backtest_results = engine.run(df.copy(), ticker)

                if backtest_results.empty:
                    continue

                metrics = MetricsCalculator.calculate_all(backtest_results, engine.ml)
                returns = backtest_results['pnl'].values

                ttest = SharpeStatistics.sharpe_ttest(returns)
                bootstrap = SharpeStatistics.bootstrap_sharpe_ci(returns, n_bootstrap=2000)

                results[factor] = {
                    'sharpe': metrics.get('Sharpe', 0.0),
                    'sortino': metrics.get('Sortino', 0.0),
                    'total_return': metrics.get('Total Return', 0.0),
                    'max_dd': metrics.get('Max Drawdown', 0.0),
                    'win_rate': metrics.get('Win Rate', 0.0),
                    'calmar': metrics.get('Calmar', 0.0),
                    'sharpe_se': ttest['se'],
                    'sharpe_tstat': ttest['t_stat'],
                    'sharpe_pvalue': ttest['p_value'],
                    'sharpe_sig_5pct': ttest['significant_5pct'],
                    'sharpe_ci_lower': bootstrap['ci_lower'],
                    'sharpe_ci_upper': bootstrap['ci_upper'],
                    'avg_weekend_adj': metrics.get('Avg Weekend Theta Adj', 1.0)
                }

                returns_by_factor[factor] = returns

                sig_marker = "✓" if ttest['significant_5pct'] else "✗"
                self.logger.success(
                    f"Factor {factor:.1f}: Sharpe={results[factor]['sharpe']:.3f} "
                    f"(t={ttest['t_stat']:.2f}, p={ttest['p_value']:.4f}) {sig_marker}"
                )

            except Exception as e:
                self.logger.error(f"Failed for factor={factor}: {e}")

        self.results = results
        self.returns_by_factor = returns_by_factor
        return results

    def find_optimal_factor(self, metric: str = 'sharpe',
                            require_significance: bool = True) -> Tuple[float, Dict]:
        """Find optimal factor."""
        if not self.results:
            return None, {}

        candidates = self.results.copy()

        if require_significance:
            candidates = {f: m for f, m in candidates.items()
                         if m.get('sharpe_sig_5pct', False)}

        if not candidates:
            candidates = self.results

        optimal_factor = max(candidates.keys(),
                            key=lambda f: candidates[f].get(metric, -999))

        return optimal_factor, candidates[optimal_factor]


# =============================================================================
# ENHANCED REPORT GENERATOR (25+ pages with new plots)
# =============================================================================

class ScientificReportGenerator:
    """
    Generate comprehensive PDF reports.

    Now includes:
    - All v3 pages (preserved)
    - NEW: ML Lift Curve
    - NEW: Turnover & Exposure analysis
    - NEW: Confusion Matrix by Regime
    """

    def __init__(self, ticker: str, df: pd.DataFrame, pdf: PdfPages,
                 config: ScientificConfig, ml_engine: MLSignalGenerator = None,
                 ml_predictions: np.ndarray = None, ml_outcomes: np.ndarray = None):
        self.ticker = ticker
        self.df = df.copy()
        self.pdf = pdf
        self.config = config
        self.ml_engine = ml_engine
        self.metrics = MetricsCalculator.calculate_all(df, ml_engine)
        self.ml_predictions = ml_predictions
        self.ml_outcomes = ml_outcomes

        if 'strategy' in self.df.columns:
            self.df['trade_id'] = (self.df['strategy'] != self.df['strategy'].shift(1)).cumsum()

    def generate(self):
        """Generate all report pages."""
        print(f"  Building report for {self.ticker}...")

        # Standard pages (light theme) - Pages 1-10
        self._page_executive_summary()
        self._page_rolling_performance()
        self._page_drawdown_analysis()
        self._page_strategy_attribution()
        self._page_regime_analysis()
        self._page_risk_decomposition()
        self._page_monthly_heatmap()
        self._page_trade_analysis()
        self._page_improvement_dashboard()
        self._page_final_summary()

        # Fixed visualizations (dark theme) - Pages 11-14
        plot_strategy_vs_buyhold(self.ticker, self.df, self.pdf)
        plot_price_with_trade_entries(self.ticker, self.df, self.pdf)
        plot_monthly_returns_line(self.ticker, self.df, self.pdf)

        # NEW: Turnover & Exposure - Page 15
        plot_turnover_exposure(self.df, self.ticker, self.pdf)

        # Reliability diagram - Page 16
        if self.ml_predictions is not None and len(self.ml_predictions) > 50:
            plot_reliability_diagram(self.ml_predictions, self.ml_outcomes,
                                    n_bins=self.config.reliability_n_bins,
                                    ticker=self.ticker, pdf=self.pdf)

        # Sharpe significance - Page 17
        returns = self.df['pnl'].values
        if len(returns) > 30:
            plot_sharpe_significance(returns, ticker=self.ticker, pdf=self.pdf)

        # NEW: ML Lift Curve - Page 18
        if self.ml_predictions is not None and len(self.ml_predictions) > 50:
            returns_aligned = self.df['ret_1'].values[:len(self.ml_predictions)]
            plot_ml_lift_curve(self.ml_predictions, self.ml_outcomes, returns_aligned,
                              ticker=self.ticker, pdf=self.pdf)

        # NEW: Confusion by Regime - Page 19
        if self.ml_predictions is not None and len(self.ml_predictions) > 50:
            plot_confusion_by_regime(self.df, self.ml_predictions, self.ml_outcomes,
                                    ticker=self.ticker, pdf=self.pdf)

    def _save(self, fig):
        self.pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

    def _page_executive_summary(self):
        """Page 1: Executive summary."""
        reset_light_theme()

        fig = plt.figure(figsize=(11, 8.5))
        gs = gridspec.GridSpec(3, 2, height_ratios=[0.5, 2, 1], hspace=0.3)

        ax_title = fig.add_subplot(gs[0, :])
        ax_title.axis('off')
        ax_title.text(0.5, 0.7, f"{self.ticker}", fontsize=28, fontweight='bold',
                     ha='center', va='center', color=LightTheme.PRIMARY)
        ax_title.text(0.5, 0.2, f"{FRAMEWORK_NAME} — Performance Report",
                     fontsize=14, ha='center', va='center', color=LightTheme.SECONDARY)

        ax_eq = fig.add_subplot(gs[1, :])
        ax_eq.plot(self.df.index, self.df['equity'], color=LightTheme.PRIMARY,
                   lw=2.5, label='Strategy')
        ax_eq.fill_between(self.df.index, self.df['equity'].iloc[0], self.df['equity'],
                          alpha=0.1, color=LightTheme.PRIMARY)
        ax_eq.axhline(self.df['equity'].iloc[0], color='gray', ls='--', alpha=0.5,
                     label='Starting Capital')
        ax_eq.set_title("Cumulative Equity", fontsize=14, fontweight='bold', pad=10)
        ax_eq.set_ylabel("Equity ($)")
        ax_eq.legend(loc='upper left')
        ax_eq.grid(True, alpha=0.3)
        ax_eq.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

        ax_met = fig.add_subplot(gs[2, :])
        ax_met.axis('off')
        m = self.metrics

        returns = self.df['pnl'].values
        ttest = SharpeStatistics.sharpe_ttest(returns)
        sharpe_sig = "✓" if ttest['significant_5pct'] else ""

        # FIX #3: Show OOS AUC
        oos_auc = m.get('ML OOS AUC (Global)', m.get('ML OOS AUC (Last)', 0.5))

        metrics_data = [
            ['Sharpe Ratio', f"{m.get('Sharpe', 0):.3f} {sharpe_sig}",
             'Sortino Ratio', f"{m.get('Sortino', 0):.3f}"],
            ['Total Return', f"{m.get('Total Return', 0):+.2%}",
             'CAGR', f"{m.get('CAGR', 0):+.2%}"],
            ['Max Drawdown', f"{m.get('Max Drawdown', 0):.2%}",
             'Win Rate', f"{m.get('Win Rate', 0):.1%}"],
            ['ML OOS AUC', f"{oos_auc:.4f}",
             'Sharpe p-value', f"{ttest['p_value']:.4f}"]
        ]
        table = ax_met.table(cellText=metrics_data, loc='center', cellLoc='center',
                            colWidths=[0.2, 0.15, 0.2, 0.15])
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1.2, 2.0)
        for i in range(4):
            table[(i, 0)].set_facecolor('#e8f4fd')
            table[(i, 2)].set_facecolor('#e8f4fd')

        self._save(fig)

    def _page_rolling_performance(self):
        """Page 2: Rolling Sharpe and Sortino."""
        reset_light_theme()

        fig, axes = plt.subplots(2, 1, figsize=(11, 8.5), sharex=True)

        roll_sharpe = self.df['pnl'].rolling(126).apply(
            lambda x: x.mean()/x.std()*np.sqrt(252) if x.std() > 0 else 0)
        axes[0].plot(self.df.index, roll_sharpe, color=LightTheme.PRIMARY, lw=2)
        axes[0].axhline(0, color='black', ls='--', alpha=0.5)
        axes[0].axhline(1.0, color=LightTheme.POSITIVE, ls='--', alpha=0.5, label='Target: 1.0')
        axes[0].fill_between(self.df.index, 0, roll_sharpe, where=roll_sharpe > 0,
                            alpha=0.3, color=LightTheme.POSITIVE)
        axes[0].fill_between(self.df.index, 0, roll_sharpe, where=roll_sharpe < 0,
                            alpha=0.3, color=LightTheme.NEGATIVE)
        axes[0].set_title("Rolling 6-Month Sharpe Ratio", fontsize=14, fontweight='bold')
        axes[0].set_ylabel("Sharpe Ratio")
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        def rolling_sortino(x):
            down = x[x < 0]
            down_std = down.std() if len(down) > 2 else x.std()
            return x.mean() / down_std * np.sqrt(252) if down_std > 0 else 0

        roll_sortino = self.df['pnl'].rolling(126).apply(rolling_sortino)
        axes[1].plot(self.df.index, roll_sortino, color=LightTheme.SECONDARY, lw=2)
        axes[1].axhline(0, color='black', ls='--', alpha=0.5)
        axes[1].axhline(1.5, color=LightTheme.POSITIVE, ls='--', alpha=0.5, label='Target: 1.5')
        axes[1].fill_between(self.df.index, 0, roll_sortino, where=roll_sortino > 0,
                            alpha=0.3, color=LightTheme.POSITIVE)
        axes[1].fill_between(self.df.index, 0, roll_sortino, where=roll_sortino < 0,
                            alpha=0.3, color=LightTheme.NEGATIVE)
        axes[1].set_title("Rolling 6-Month Sortino Ratio", fontsize=14, fontweight='bold')
        axes[1].set_ylabel("Sortino Ratio")
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        self._save(fig)

    def _page_drawdown_analysis(self):
        """Page 3: Drawdown analysis."""
        reset_light_theme()

        fig, axes = plt.subplots(2, 1, figsize=(11, 8.5))

        dd = self.df['drawdown'] * 100
        axes[0].fill_between(self.df.index, 0, dd, color=LightTheme.NEGATIVE, alpha=0.7)
        axes[0].plot(self.df.index, dd, color='darkred', lw=1)
        axes[0].axhline(-10, color='orange', ls='--', alpha=0.7, label='Warning: -10%')
        axes[0].set_title("Drawdown Over Time", fontsize=14, fontweight='bold')
        axes[0].set_ylabel("Drawdown (%)")
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        axes[1].hist(dd, bins=50, color=LightTheme.NEGATIVE, alpha=0.7, edgecolor='darkred')
        axes[1].axvline(dd.quantile(0.05), color='black', ls='--', lw=2,
                       label=f'5th Percentile: {dd.quantile(0.05):.1f}%')
        axes[1].set_title("Drawdown Distribution", fontsize=14, fontweight='bold')
        axes[1].set_xlabel("Drawdown (%)")
        axes[1].set_ylabel("Frequency")
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        self._save(fig)

    def _page_strategy_attribution(self):
        """Page 4: Strategy allocation and P&L."""
        reset_light_theme()

        fig, axes = plt.subplots(1, 2, figsize=(11, 8.5))

        if 'strategy' in self.df.columns:
            strat_counts = self.df['strategy'].value_counts()
            colors_list = [LightTheme.PRIMARY, LightTheme.SECONDARY, LightTheme.ACCENT,
                          LightTheme.POSITIVE, LightTheme.WARNING, LightTheme.NEGATIVE][:len(strat_counts)]
            axes[0].pie(strat_counts.values, labels=strat_counts.index, autopct='%1.1f%%',
                       colors=colors_list, startangle=90)
            axes[0].set_title("Strategy Allocation", fontsize=14, fontweight='bold')

            strat_pnl = self.df.groupby('strategy')['pnl'].sum().sort_values(ascending=True)
            bar_colors = [LightTheme.POSITIVE if x > 0 else LightTheme.NEGATIVE
                         for x in strat_pnl.values]
            axes[1].barh(strat_pnl.index, strat_pnl.values * 100, color=bar_colors)
            axes[1].axvline(0, color='black', lw=1)
            axes[1].set_title("P&L by Strategy (%)", fontsize=14, fontweight='bold')
            axes[1].set_xlabel("Cumulative P&L (%)")
            axes[1].grid(True, alpha=0.3, axis='x')

        plt.tight_layout()
        self._save(fig)

    def _page_regime_analysis(self):
        """Page 5: Regime detection (now without leakage)."""
        reset_light_theme()

        fig = plt.figure(figsize=(11, 8.5))
        gs = gridspec.GridSpec(2, 2, hspace=0.3, wspace=0.3)

        if 'regime' not in self.df.columns:
            ax = fig.add_subplot(gs[:, :])
            ax.text(0.5, 0.5, "No regime data available", ha='center', va='center', fontsize=14)
            ax.axis('off')
            self._save(fig)
            return

        ax1 = fig.add_subplot(gs[0, :])
        regime_map = {'Bull': 3, 'Neutral': 2, 'Bear': 1, 'Crisis': 0}
        regime_numeric = self.df['regime'].map(regime_map).fillna(2)
        ax1.fill_between(self.df.index, 0, regime_numeric, alpha=0.5,
                        color=LightTheme.PRIMARY, step='post')
        ax1.set_yticks([0, 1, 2, 3])
        ax1.set_yticklabels(['CRISIS', 'BEAR', 'NEUTRAL', 'BULL'])
        ax1.set_title("Market Regime (HMM - No Leakage in v4)", fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)

        ax2 = fig.add_subplot(gs[1, 0])
        regime_counts = self.df['regime'].value_counts()
        color_map = {'Bull': LightTheme.POSITIVE, 'Neutral': LightTheme.WARNING,
                    'Bear': LightTheme.NEGATIVE, 'Crisis': 'darkred'}
        bar_colors = [color_map.get(r, 'gray') for r in regime_counts.index]
        ax2.bar(regime_counts.index, regime_counts.values, color=bar_colors)
        ax2.set_title("Regime Distribution", fontsize=12, fontweight='bold')
        ax2.set_ylabel("Days")

        ax3 = fig.add_subplot(gs[1, 1])
        regime_pnl = self.df.groupby('regime')['pnl'].agg(['mean', 'std'])
        regime_pnl['sharpe'] = regime_pnl['mean'] / regime_pnl['std'] * np.sqrt(252)
        x = range(len(regime_pnl))
        bar_colors = [color_map.get(r, 'gray') for r in regime_pnl.index]
        ax3.bar(x, regime_pnl['sharpe'], color=bar_colors)
        ax3.set_xticks(x)
        ax3.set_xticklabels(regime_pnl.index)
        ax3.axhline(0, color='black', lw=1)
        ax3.set_title("Sharpe by Regime", fontsize=12, fontweight='bold')
        ax3.set_ylabel("Sharpe Ratio")
        ax3.grid(True, alpha=0.3, axis='y')

        self._save(fig)

    def _page_risk_decomposition(self):
        """Page 6: Risk metrics."""
        reset_light_theme()

        fig, axes = plt.subplots(2, 2, figsize=(11, 8.5))

        r = self.df['pnl']

        axes[0, 0].hist(r * 100, bins=50, color=LightTheme.PRIMARY, alpha=0.7, edgecolor='white')
        axes[0, 0].axvline(r.mean() * 100, color=LightTheme.POSITIVE, lw=2, ls='--',
                          label=f'Mean: {r.mean()*100:.3f}%')
        axes[0, 0].axvline(r.quantile(0.05) * 100, color=LightTheme.NEGATIVE, lw=2, ls='--',
                          label=f'VaR 95%: {r.quantile(0.05)*100:.3f}%')
        axes[0, 0].set_title("Daily P&L Distribution", fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel("Daily P&L (%)")
        axes[0, 0].legend(fontsize=8)
        axes[0, 0].grid(True, alpha=0.3)

        probplot(r, dist="norm", plot=axes[0, 1])
        axes[0, 1].set_title("Q-Q Plot vs Normal", fontsize=12, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)

        roll_vol = r.rolling(20).std() * np.sqrt(252) * 100
        axes[1, 0].plot(self.df.index, roll_vol, color=LightTheme.PRIMARY, lw=1.5)
        axes[1, 0].axhline(12, color=LightTheme.POSITIVE, ls='--', label='Target: 12%')
        axes[1, 0].fill_between(self.df.index, roll_vol, 12, where=roll_vol > 12,
                               alpha=0.3, color=LightTheme.NEGATIVE)
        axes[1, 0].fill_between(self.df.index, roll_vol, 12, where=roll_vol < 12,
                               alpha=0.3, color=LightTheme.POSITIVE)
        axes[1, 0].set_title("Rolling Volatility (ART)", fontsize=12, fontweight='bold')
        axes[1, 0].set_ylabel("Annualized Vol (%)")
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        axes[1, 1].axis('off')
        m = self.metrics
        risk_data = [
            ['Metric', 'Value'],
            ['VaR 95%', f"{m.get('VaR 95%', 0)*100:.3f}%"],
            ['ES 95%', f"{m.get('ES 95%', 0)*100:.3f}%"],
            ['Skewness', f"{m.get('Skewness', 0):.3f}"],
            ['Kurtosis', f"{m.get('Kurtosis', 0):.3f}"],
            ['Omega Ratio', f"{m.get('Omega', 0):.3f}"],
            ['Profit Factor', f"{m.get('Profit Factor', 0):.3f}"]
        ]
        table = axes[1, 1].table(cellText=risk_data, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1.2, 2.0)

        plt.tight_layout()
        self._save(fig)

    def _page_monthly_heatmap(self):
        """Page 7: Monthly returns heatmap."""
        reset_light_theme()

        fig, ax = plt.subplots(figsize=(11, 8.5))

        monthly = self.df['pnl'].resample('M').sum() * 100
        monthly_df = pd.DataFrame({
            'Year': monthly.index.year,
            'Month': monthly.index.month,
            'Return': monthly.values
        })
        pivot = monthly_df.pivot(index='Year', columns='Month', values='Return')
        pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]

        sns.heatmap(pivot, annot=True, fmt='.1f', cmap='RdYlGn', center=0, ax=ax,
                   cbar_kws={'label': 'Monthly Return (%)'})
        ax.set_title("Monthly Returns Heatmap (%)", fontsize=14, fontweight='bold')

        plt.tight_layout()
        self._save(fig)

    def _page_trade_analysis(self):
        """Page 8: Top trades."""
        reset_light_theme()

        fig, ax = plt.subplots(figsize=(11, 8.5))
        ax.axis('off')

        if 'trade_id' not in self.df.columns:
            ax.text(0.5, 0.5, "No trade data", ha='center', va='center', fontsize=14)
            self._save(fig)
            return

        trades = self.df.groupby(['trade_id', 'strategy'])['pnl'].sum().reset_index()
        trades = trades[trades['strategy'] != 'Cash'].sort_values('pnl', ascending=False)

        if len(trades) == 0:
            ax.text(0.5, 0.5, "No trades executed", ha='center', va='center', fontsize=14)
            self._save(fig)
            return

        top10 = trades.head(10)
        bottom10 = trades.tail(10)
        combined = pd.concat([top10, bottom10])

        table_data = [['Rank', 'Strategy', 'P&L (%)']]
        for i, (_, row) in enumerate(combined.iterrows()):
            rank = i + 1 if i < 10 else f"B{i-9}"
            table_data.append([rank, row['strategy'], f"{row['pnl']*100:+.4f}%"])

        table = ax.table(cellText=table_data, loc='center', cellLoc='center',
                        colWidths=[0.15, 0.45, 0.2])
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.1, 1.8)

        ax.set_title("Top 10 Winners & Losers", fontsize=16, fontweight='bold', pad=20)
        self._save(fig)

    def _page_improvement_dashboard(self):
        """Page 9: Improvements dashboard."""
        reset_light_theme()

        fig = plt.figure(figsize=(11, 8.5))
        ax = fig.add_subplot(111)
        ax.axis('off')

        m = self.metrics
        oos_auc = m.get('ML OOS AUC (Global)', m.get('ML OOS AUC (Mean Window)', 0.5))

        improvements = [
            ('1. Robust ML (Lasso Logistic)', f"OOS AUC: {oos_auc:.4f}",
             '✅' if oos_auc > 0.52 else '⚠️'),
            ('2. HMM Regime (No Leakage v4)', "Fixed: Train-only fitting", '✅'),
            ('3. Parkinson/GK Vol Estimators', "Using High-Low data", '✅'),
            ('4. Vol Percentile (renamed)', "Accurate nomenclature", '✅'),
            ('5. Auto-Regressive Risk Targeting', f"Realized: {m.get('Realized Vol', 0)*100:.1f}%",
             '✅' if m.get('Vol Target Hit', False) else '⚠️'),
            ('6. Weekend Theta WIRED', f"Avg Adj: {m.get('Avg Weekend Theta Adj', 1.0):.4f}", '✅'),
            ('7. Strategy Leverage Diff', "Per-strategy scaling", '✅'),
            ('8. OOS AUC Reporting', "Test AUC, not train", '✅')
        ]

        table_data = [['Improvement', 'Metric', 'Status']] + [list(imp) for imp in improvements]
        table = ax.table(cellText=table_data, loc='center', cellLoc='left',
                        colWidths=[0.4, 0.35, 0.1])
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1.1, 2.2)

        ax.set_title(f"v{FRAMEWORK_VERSION} Fixes & Improvements Dashboard", fontsize=16, fontweight='bold', y=0.95)
        self._save(fig)

    def _page_final_summary(self):
        """Page 10: Final summary."""
        reset_light_theme()

        fig, ax = plt.subplots(figsize=(11, 8.5))
        ax.axis('off')

        m = self.metrics
        sharpe = m.get('Sharpe', 0)

        returns = self.df['pnl'].values
        ttest = SharpeStatistics.sharpe_ttest(returns)
        bootstrap = SharpeStatistics.bootstrap_sharpe_ci(returns)

        if sharpe > 1.0 and ttest['significant_5pct']:
            perf_grade = "EXCELLENT (Significant)"
        elif sharpe > 0.5 and ttest['significant_5pct']:
            perf_grade = "GOOD (Significant)"
        elif sharpe > 0.5:
            perf_grade = "DEVELOPING"
        elif sharpe > 0:
            perf_grade = "WEAK POSITIVE"
        else:
            perf_grade = "NEEDS IMPROVEMENT"

        summary_text = f"""
        {self.ticker} - FINAL SUMMARY (v{FRAMEWORK_VERSION})

        Core Metrics:
        - Sharpe Ratio:   {sharpe:.4f}
        - Sortino Ratio:  {m.get('Sortino', 0):.4f}
        - Total Return:   {m.get('Total Return', 0):+.2%}
        - Max Drawdown:   {m.get('Max Drawdown', 0):.2%}

        Statistical Significance:
        - t-stat:         {ttest['t_stat']:.4f}
        - p-value:        {ttest['p_value']:.6f}
        - 95% CI:         [{bootstrap['ci_lower']:.4f}, {bootstrap['ci_upper']:.4f}]
        - Significant:    {'Yes' if ttest['significant_5pct'] else 'No'}

        ML Performance (OOS):
        - Global AUC:     {m.get('ML OOS AUC (Global)', 0.5):.4f}

        GRADE: {perf_grade}
        """

        ax.text(0.5, 0.5, summary_text.strip(), ha='center', va='center', fontsize=13,
               fontfamily='monospace',
               bbox=dict(facecolor=LightTheme.PRIMARY, alpha=0.95, boxstyle='round,pad=0.5'),
               color='white')

        self._save(fig)


# =============================================================================
# PORTFOLIO SUMMARY GENERATOR (Preserved)
# =============================================================================

class PortfolioSummaryGenerator:
    """Generate portfolio-level summary."""

    def __init__(self, portfolio_metrics: List[Dict], all_results: Dict[str, pd.DataFrame],
                 pdf: PdfPages):
        self.metrics = pd.DataFrame(portfolio_metrics)
        self.all_results = all_results
        self.pdf = pdf

    def generate(self):
        self._page_summary_table()
        self._page_comparison_charts()
        plot_cross_asset_comparison(self.all_results, self.pdf)

    def _page_summary_table(self):
        reset_light_theme()

        fig, ax = plt.subplots(figsize=(13.69, 8.27))
        ax.axis('off')

        ax.text(0.5, 0.96, "Portfolio Performance Summary", ha='center', fontsize=24,
               fontweight='bold', transform=ax.transAxes, color=LightTheme.PRIMARY)
        ax.text(0.5, 0.91, "Scientific Options Framework v4.2 — All Assets", ha='center',
               fontsize=12, transform=ax.transAxes, color=LightTheme.SECONDARY)

        show = self.metrics.copy()

        # Format columns that exist (handle missing gracefully)
        if 'Return' in show.columns:
            show['Return'] = show['Return'].map('{:+.2%}'.format)
        if 'Sharpe' in show.columns:
            show['Sharpe'] = show['Sharpe'].map('{:.4f}'.format)
        if 'Sortino' in show.columns:
            show['Sortino'] = show['Sortino'].map('{:.4f}'.format)
        if 'MaxDD' in show.columns:
            show['MaxDD'] = show['MaxDD'].map('{:.2%}'.format)
        if 'WinRate' in show.columns:
            show['WinRate'] = show['WinRate'].map('{:.1%}'.format)
        if 'OOS_AUC' in show.columns:
            show['OOS_AUC'] = show['OOS_AUC'].map('{:.4f}'.format)
        if 'VaR_Loss' in show.columns:
            show['VaR_Loss'] = show['VaR_Loss'].map('{:.4f}'.format)
        if 'ES_Loss' in show.columns:
            show['ES_Loss'] = show['ES_Loss'].map('{:.4f}'.format)
        if 'SharpeSig' in show.columns:
            show['SharpeSig'] = show['SharpeSig'].map(lambda x: '✓' if x else '')

        # Build column list from what's available
        possible_cols = ['Asset', 'Return', 'Sharpe', 'Sortino', 'MaxDD', 'WinRate',
                         'OOS_AUC', 'VaR_Loss', 'ES_Loss', 'SharpeSig']
        cols = [c for c in possible_cols if c in show.columns]
        show = show[cols]

        table_data = [show.columns.tolist()] + show.values.tolist()
        table = ax.table(cellText=table_data, loc='center', bbox=[0.05, 0.20, 0.90, 0.65])
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2.2)

        for j in range(len(show.columns)):
            table[(0, j)].set_facecolor(LightTheme.PRIMARY)
            table[(0, j)].get_text().set_color('white')

        self.pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

    def _page_comparison_charts(self):
        reset_light_theme()

        fig, axes = plt.subplots(2, 2, figsize=(13.69, 8.27))

        # Sharpe chart (always available)
        colors_pos_neg = [LightTheme.POSITIVE if x > 0 else LightTheme.NEGATIVE
                        for x in self.metrics['Sharpe']]
        axes[0, 0].barh(self.metrics['Asset'], self.metrics['Sharpe'], color=colors_pos_neg)
        axes[0, 0].axvline(0, color='black', lw=1)
        axes[0, 0].set_title("Sharpe Ratio by Asset", fontsize=12, fontweight='bold')
        axes[0, 0].grid(True, alpha=0.3, axis='x')

        # OOS AUC or Sortino (if available)
        if 'OOS_AUC' in self.metrics.columns:
            axes[0, 1].barh(self.metrics['Asset'], self.metrics['OOS_AUC'],
                           color=LightTheme.SECONDARY)
            axes[0, 1].axvline(0.5, color='black', ls='--', alpha=0.5)
            axes[0, 1].set_title("ML OOS AUC by Asset", fontsize=12, fontweight='bold')
        elif 'Sortino' in self.metrics.columns:
            colors_pos_neg = [LightTheme.POSITIVE if x > 0 else LightTheme.NEGATIVE
                            for x in self.metrics['Sortino']]
            axes[0, 1].barh(self.metrics['Asset'], self.metrics['Sortino'], color=colors_pos_neg)
            axes[0, 1].axvline(0, color='black', lw=1)
            axes[0, 1].set_title("Sortino Ratio by Asset", fontsize=12, fontweight='bold')
        else:
            axes[0, 1].text(0.5, 0.5, 'No additional ratio data', ha='center', va='center')
            axes[0, 1].set_title("Additional Metrics", fontsize=12, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3, axis='x')

        # Returns chart
        colors_pos_neg = [LightTheme.POSITIVE if x > 0 else LightTheme.NEGATIVE
                        for x in self.metrics['Return']]
        axes[1, 0].barh(self.metrics['Asset'], self.metrics['Return'] * 100, color=colors_pos_neg)
        axes[1, 0].axvline(0, color='black', lw=1)
        axes[1, 0].set_title("Total Return (%) by Asset", fontsize=12, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3, axis='x')

        # VaR Loss or WinRate
        if 'VaR_Loss' in self.metrics.columns:
            axes[1, 1].barh(self.metrics['Asset'], self.metrics['VaR_Loss'] * 100,
                           color=LightTheme.NEGATIVE)
            axes[1, 1].set_title("VaR 95% Loss (%) by Asset", fontsize=12, fontweight='bold')
        elif 'WinRate' in self.metrics.columns:
            axes[1, 1].barh(self.metrics['Asset'], self.metrics['WinRate'] * 100,
                           color=LightTheme.SECONDARY)
            axes[1, 1].axvline(50, color='black', ls='--', alpha=0.5)
            axes[1, 1].set_title("Win Rate (%) by Asset", fontsize=12, fontweight='bold')
        else:
            axes[1, 1].text(0.5, 0.5, 'No risk data', ha='center', va='center')
            axes[1, 1].set_title("Risk Metrics", fontsize=12, fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3, axis='x')

        plt.suptitle("Cross-Asset Performance", fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        self.pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)


# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main execution function."""
    print("=" * 80)
    print(FRAMEWORK_NAME)
    print("=" * 80)
    print()
    print("FIXES APPLIED (v4.0 → v4.2):")
    print("  ✅ HMM: Fits on train only, predicts on test (no leakage)")
    print("  ✅ ML: Reports OOS AUC (not train), drops last training row")
    print("  ✅ Weekend Theta: Actually affects PnL on Fridays")
    print("  ✅ Strategy Leverage: Different scaling per strategy type")
    print("  ✅ Regime Mapping: States named by mean return (Bull/Neutral/Bear)")
    print("  ✅ Vol Percentile: Renamed from VRP, uses searchsorted")
    print("  ✅ VaR/ES: Clear sign conventions + loss versions")
    print()
    print("=" * 80)

    config = ScientificConfig()
    config.logging.level = LogLevel.NORMAL
    config.train_window = 252
    config.test_window = 21

    TOP_ASSETS = [
        "AUDUSD","GBPUSD","USA500.IDXUSD", "XAUUSD", "BTCUSD", "USATECH.IDXUSD",
        "EURUSD", "DIESEL","USTBOND"
    ]

    loader = DataLoader(config)
    logger = Logger(config.logging)
    portfolio_metrics = []
    all_results = {}

    for ticker in TOP_ASSETS:
        logger.header(f"Processing: {ticker}")

        try:
            df = loader.load(ticker)
            logger.info(f"Loaded {len(df):,} daily bars")

            engine = ScientificStrategyEngine(config)
            results = engine.run(df, ticker)

            if results.empty:
                logger.warning(f"No results for {ticker}")
                continue

            all_results[ticker] = results.copy()
            metrics = MetricsCalculator.calculate_all(results, engine.ml)

            ml_preds, ml_outcomes = engine.get_ml_calibration_data()

            returns = results['pnl'].values
            ttest = SharpeStatistics.sharpe_ttest(returns)
            bootstrap = SharpeStatistics.bootstrap_sharpe_ci(returns, n_bootstrap=2000)

            logger.info(f"\nRESULTS:")
            logger.info(f"├─ Sharpe:        {metrics.get('Sharpe', 0):.4f}")
            logger.info(f"├─ t-stat:        {ttest['t_stat']:.4f}")
            logger.info(f"├─ p-value:       {ttest['p_value']:.6f}")
            logger.info(f"├─ ML OOS AUC:    {engine.get_ml_global_oos_auc():.4f}")
            logger.info(f"├─ Total Return:  {metrics.get('Total Return', 0):+.2%}")
            logger.info(f"└─ Max Drawdown:  {metrics.get('Max Drawdown', 0):.2%}")

            # Generate report - use version in filename
            report_path = config.report_dir / f"{ticker.replace('.', '_')}_Report_v{FRAMEWORK_VERSION.replace('.', '_')}.pdf"
            with PdfPages(report_path) as pdf:
                report = ScientificReportGenerator(
                    ticker, results, pdf, config,
                    ml_engine=engine.ml,
                    ml_predictions=ml_preds, ml_outcomes=ml_outcomes
                )
                report.generate()

            logger.success(f"Report saved: {report_path.name}")

            portfolio_metrics.append({
                'Asset': ticker,
                'Return': metrics.get('Total Return', 0),
                'Sharpe': metrics.get('Sharpe', 0),
                'Sortino': metrics.get('Sortino', 0),
                'MaxDD': metrics.get('Max Drawdown', 0),
                'WinRate': metrics.get('Win Rate', 0),
                'OOS_AUC': engine.get_ml_global_oos_auc(),
                'SharpeSig': ttest['significant_5pct'],
                'SharpePval': ttest['p_value'],
                'SharpeCI_L': bootstrap['ci_lower'],
                'SharpeCI_U': bootstrap['ci_upper']
            })

        except Exception as e:
            logger.error(f"Error: {e}")
            import traceback
            traceback.print_exc()

    # Portfolio summary
    if portfolio_metrics:
        logger.header("PORTFOLIO SUMMARY")

        summary_path = config.report_dir / f"Portfolio_Summary_v{FRAMEWORK_VERSION.replace('.', '_')}.pdf"
        with PdfPages(summary_path) as pdf:
            summary = PortfolioSummaryGenerator(portfolio_metrics, all_results, pdf)
            summary.generate()

        logger.success(f"Portfolio Summary saved: {summary_path.name}")

        df_summary = pd.DataFrame(portfolio_metrics)
        print("\n" + "=" * 100)
        print(f"FINAL PORTFOLIO METRICS (v{FRAMEWORK_VERSION})")
        print("=" * 100)

        display_cols = ['Asset', 'Sharpe', 'OOS_AUC', 'SharpeSig', 'Return', 'MaxDD']
        print(df_summary[display_cols].to_string(index=False))

        print(f"\nAverage Sharpe:        {df_summary['Sharpe'].mean():.4f}")
        print(f"Average OOS AUC:       {df_summary['OOS_AUC'].mean():.4f}")
        print(f"Significant (α=0.05):  {df_summary['SharpeSig'].sum()} / {len(df_summary)}")

        csv_path = config.report_dir / f"portfolio_metrics_v{FRAMEWORK_VERSION.replace('.', '_')}.csv"
        df_summary.to_csv(csv_path, index=False)

    print("\n" + "=" * 80)
    print(f"✅ {FRAMEWORK_NAME} COMPLETE!")
    print("=" * 80)
    print(f"\n📁 Reports: {config.report_dir}")

    logger.close()


if __name__ == "__main__":
    main()

Scientific Options Framework v4.9

FIXES APPLIED (v4.0 → v4.2):
  ✅ HMM: Fits on train only, predicts on test (no leakage)
  ✅ ML: Reports OOS AUC (not train), drops last training row
  ✅ Weekend Theta: Actually affects PnL on Fridays
  ✅ Strategy Leverage: Different scaling per strategy type
  ✅ Regime Mapping: States named by mean return (Bull/Neutral/Bear)
  ✅ Vol Percentile: Renamed from VRP, uses searchsorted
  ✅ VaR/ES: Clear sign conventions + loss versions


Processing: AUDUSD
ℹ Loaded 1,564 daily bars
Walk-forward [████████████████████████████████████████] 100.0% (59/59)
ℹ 
RESULTS:
ℹ ├─ Sharpe:        3.5312
ℹ ├─ t-stat:        22.8785
ℹ ├─ p-value:       0.000000
ℹ ├─ ML OOS AUC:    0.5436
ℹ ├─ Total Return:  +8.13%
ℹ └─ Max Drawdown:  -0.24%
  Building report for AUDUSD...
✓ Report saved: AUDUSD_Report_v4_9.pdf

Processing: GBPUSD
ℹ Loaded 1,563 daily bars
Walk-forward [████████████████████████████████████████] 100.0% (59/59)
ℹ 
RESULTS:
ℹ ├─ Sharpe:        4.6170
ℹ ├─ t-st